In [ ]:
#验证数据集的处理

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge  # 使用贝叶斯岭回归
print(os.getcwd())
datapath='../data_val'

C:\Users\shijuan_125623147720\Desktop\ruijin\Deliver_v1


In [3]:
df_v=pd.read_csv(os.path.join(datapath, 'df_val_visit.csv'))
df_d=pd.read_csv(os.path.join(datapath, 'df_val_drug.csv'))
print('user_id:',df_v['user_id'].nunique(),df_v[df_v['age']>=18]['user_id'].nunique())
print(df_v['hosp_id'].nunique())
df_v.info()

user_id: 38545 38545
49
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106610 entries, 0 to 106609
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            106610 non-null  int64  
 1   visit_level        106610 non-null  int64  
 2   visit_at           106610 non-null  object 
 3   hosp_id            106610 non-null  int64  
 4   age                106610 non-null  int64  
 5   diabetic_duration  103712 non-null  float64
 6   sbp                78670 non-null   float64
 7   dbp                78664 non-null   float64
 8   bmi_v1             78408 non-null   float64
 9   ALT                94940 non-null   float64
 10  LDL_c              93962 non-null   float64
 11  Cr                 94609 non-null   float64
 12  FPG                100048 non-null  float64
 13  CP0                64239 non-null   float64
 14  HbA1c              106610 non-null  float64
 15  sex                106610 n

In [4]:
#医院减少到4家
df_v= df_v[(df_v['hosp_id'] == 11362) | (df_v['hosp_id'] == 10695) | (df_v['hosp_id'] == 13619) | (df_v['hosp_id'] == 12105)]
print('user_id:',df_v['user_id'].nunique(),df_v[df_v['age']>=18]['user_id'].nunique())
# count_user_id(df_v)

user_id: 13773 13773
user_id: 13773


KeyError: 'type_no_class'

In [3]:
print(df_d.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247955 entries, 0 to 247954
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        247955 non-null  int64  
 1   drug_type      247955 non-null  int64  
 2   drug_no        247955 non-null  int64  
 3   type_no_class  224970 non-null  float64
 4   start_date     247955 non-null  object 
 5   end_date       247955 non-null  object 
 6   adjust_date    247955 non-null  object 
 7   flag_drug      120869 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 15.1+ MB
None


In [1]:
def count_user_id(df):
    print('user_id:',df['user_id'].nunique())
    print(df[df['type_no_class'] == 1]['user_id'].nunique())
    print(df[df['type_no_class'] == 2]['user_id'].nunique())

In [8]:
def drug_type(df2):
    df2['start_date'] = pd.to_datetime(df2['start_date'], errors='coerce')
    # df2_1  = df2[df2['flag_drug'] != 1]
    df2_1=df2.copy()
    # 将 'type_no_class' 列转换为数值型
    df2_1.loc[:, 'type_no_class'] = pd.to_numeric(df2_1['type_no_class'], errors='coerce')

    # 将 'type_no_class' 列中的空值转换为0
    df2_1.loc[:, 'type_no_class'] = df2_1['type_no_class'].fillna(1).astype('Int64')
    # 对特定值应用映射
    # df2_1.loc[df2_1['type_no_class'].between(9, 18), 'type_no_class'] = -10
    #如果drug_no为68001或51001，将type_no_class列中的值替换为-10
    df2_1.loc[df2_1['drug_no'].isin([68001, 51001]), 'type_no_class'] = -10
    #TODO：药物编号51001中有关键字“胰岛素”，药物编号51001中有关键字“优思灵/林”、“万秀林”均属于胰岛素，只要使用了任意一种就视为使用胰岛素
    df2_1.loc[:, 'type_no_class'] = df2_1['type_no_class'].replace({6: -1, 8: -2, 2: 0})

    df2_2 = df2_1[df2_1['type_no_class'].isin([-1, -2, 0, 9,10,11,12,13,14,15,16,17,18,-10])]
    # df2_2.drop(['general name', 'drug_no', 'type_category','flag_drug'], axis=1, inplace=True)
    df2_2 = df2_2.drop([ 'drug_no', 'flag_drug'], axis=1)

    df2_2.loc[:, 'type_no_class'] = df2_2['type_no_class'].replace({-1: 1, -2: 2, -10: 19})
    return df2_2
def adjust_drug_type(df):
    '''
    删除所有drug_type=2的行
    将所有drug_type=4的行的drug_type改为1
    对于所有drug_type=1的行，如果存在另一行满足user_id、start_date、type_no_class相同且drug_type=3，则删除这个drug_type=1的行
    '''
    df = df[df['drug_type'] != 2]
    df.loc[df['drug_type'] == 4, 'drug_type'] = 1
     # 初始化一个空的索引列表用于记录需要删除的行
    rows_to_remove = []

    # 遍历数据帧，检查每一对相邻的行
    for i in range(len(df) - 1):
        current_row = df.iloc[i]
        next_row = df.iloc[i + 1]

        # 如果当前行和下一行的user_id, start_date, type_no_class相同
        # 并且当前行是drug_type=1，下一行是drug_type=3，那么删除当前行
        if (current_row['user_id'] == next_row['user_id'] and
            current_row['start_date'] == next_row['start_date'] and
            current_row['type_no_class'] == next_row['type_no_class'] and
            current_row['drug_type'] == 1 and next_row['drug_type'] == 3):
            rows_to_remove.append(i)

    # 删除需要删除的行
    df = df.drop(df.index[rows_to_remove])

    return df


In [9]:
df2_1 = drug_type(df_d)
df2_1.sort_values(by=['user_id',  'type_no_class','start_date','drug_type'], inplace=True)
df2_2=adjust_drug_type(df2_1)
df2_2.head()

,user_id,drug_type,type_no_class,start_date,end_date,adjust_date
46,20071,1,0.0,2017-03-17,0001-01-01,0001-01-01
132,20071,1,2.0,2022-10-24,0001-01-01,0001-01-01
5,20071,1,17.0,2022-10-24,0001-01-01,0001-01-01
306,854897,1,0.0,2010-01-01,0001-01-01,0001-01-01
500,854897,1,2.0,2024-05-17,0001-01-01,0001-01-01


In [10]:
def merge_period(df):
    '''
    合并服药时间:
    遍历所有行，如果下一行和当前行的user_id、type_no_class相同，且下一行的start_date等于当前行的end_date，则将下一行的start_date改为当前行的start_date，并删除当前行
    '''
    df=df.copy()
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
    # 初始化一个空的索引列表用于记录需要删除的行
    rows_to_remove = []
    #遍历所有行，如果下一行和当前行的user_id、type_no_class相同，且下一行的start_date等于当前行的end_date，则将下一行的start_date改为当前行的start_date，并删除当前行
    for i in range(len(df) - 1):
        current_row = df.iloc[i]
        next_row = df.iloc[i + 1]
        if (current_row['user_id'] == next_row['user_id'] and
                current_row['type_no_class'] == next_row['type_no_class'] and
                (next_row['start_date'] - current_row['end_date']).days >= 0 and
                #或者next_row['start_date']只比current_row['end_date']大不超过30天
                 (next_row['start_date'] - current_row['end_date']).days <= 30):
            df.at[i + 1, 'T0'] = current_row['start_date']#这里先定义了T0，不过没影响，后面还会重新定义
            df.at[i + 1, 'start_date'] = current_row['start_date']
            rows_to_remove.append(i)
    return df.drop(df.index[rows_to_remove])


In [11]:
print(df2_2.shape)
df2_21=merge_period(df2_2)
print(df2_21.shape)

(71579, 6)


C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\2170195809.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')


(70810, 7)


In [12]:
def jia_time(df):
    df=df.copy()
    df['T0'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')

    # 初始化 jia_flag 和 jia_time
    df['jia_flag'] = 0# 二甲双胍是否满足
    df['jia_time'] = 0# T0之前的1个月期间二甲双胍使用时间

    for user_id, group in df.groupby('user_id'):
        # 预先筛选二甲双胍使用记录
        metformin_usage = group[group['type_no_class'] == 0]
        for index, row in group[group['type_no_class'].isin([1, 2])].iterrows():
            T0 = row['T0']
            start_date = T0 - pd.Timedelta(days=30)
            end_date = T0

            # 筛选出在 T0 前一个月的二甲双胍使用记录
            valid_usage = metformin_usage[(metformin_usage['start_date'] < end_date) &
                                          ((metformin_usage['end_date'] >= start_date) |
                                           (metformin_usage['end_date'].isna()))]

            # 计算有效使用天数
            covered_days = set()
            for _, m_row in valid_usage.iterrows():
                metformin_start = max(m_row['start_date'], start_date)
                metformin_end = min(m_row['end_date'] if pd.notna(m_row['end_date']) else end_date, end_date)
                covered_days.update(pd.date_range(start=metformin_start, end=metformin_end).date)

            df.at[index, 'jia_time'] = len(covered_days)
            df.at[index, 'jia_flag'] = 1 if len(covered_days) >= 23 else 0
    return df

# df2_2 = df2_2.drop(['drug_type','adjust_date', 'is_used'], axis=1)
df2_3 = jia_time(df2_21)#计算二甲双胍
print(df2_3['jia_flag'].value_counts())
df2_3.shape

jia_flag
0    60971
1     9839
Name: count, dtype: int64


(70810, 9)

In [13]:
df2_3.head(5)

,user_id,drug_type,type_no_class,start_date,end_date,adjust_date,T0,jia_flag,jia_time
46,20071.0,1.0,0.0,2017-03-17,NaT,0001-01-01,2017-03-17,0,0
132,20071.0,1.0,2.0,2022-10-24,NaT,0001-01-01,2022-10-24,1,31
5,20071.0,1.0,17.0,2022-10-24,NaT,0001-01-01,2022-10-24,0,0
306,854897.0,1.0,0.0,2010-01-01,NaT,0001-01-01,2010-01-01,0,0
500,854897.0,1.0,2.0,2024-05-17,NaT,0001-01-01,2024-05-17,1,31


In [14]:
def yi_time(df):
    df=df.copy()
    df['T0'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
    df.loc[:, 'yi_flag'] = 0  # 胰岛素是否满足，1满足，0不满足
    df.loc[:, 'yi_time'] = 0  # 胰岛素是否满足

    for user_id, group in df.groupby('user_id'):
        # 预先筛选出胰岛素使用记录（type_no_class 介于 9 到 19 之间）
        insulin_usage = group[group['type_no_class'].between(9, 19)]#搜索范围是左闭右闭
        for index, row in group[group['type_no_class'].isin([1, 2])].iterrows():
            T0 = row['T0']
            start_date = T0 - pd.Timedelta(days=90)
            end_date = T0

            # 筛选出在 T0 前90天内的胰岛素使用记录
            valid_usage = insulin_usage[(insulin_usage['start_date'] < end_date) &
                                        ((insulin_usage['end_date'] >= start_date) |
                                         (insulin_usage['end_date'].isna()))]

            # 计算有效使用天数（考虑不同胰岛素的使用时间并集）
            covered_days = set()
            for _, i_row in valid_usage.iterrows():
                insulin_start = max(i_row['start_date'], start_date)
                insulin_end = min(i_row['end_date'] if pd.notna(i_row['end_date']) else end_date, end_date)
                covered_days.update(pd.date_range(start=insulin_start, end=insulin_end).date)

            df.at[index, 'yi_time'] = len(covered_days)
            df.at[index, 'yi_flag'] = 0 if  len(covered_days) >= 30 else 1

    return df

df2_4 = yi_time(df2_3)
df2_4.to_csv(os.path.join(datapath, 'df2_4.csv'), index=False)
df2_4.shape

(70810, 11)

In [15]:
df2_4.head()

,user_id,drug_type,type_no_class,start_date,end_date,adjust_date,T0,jia_flag,jia_time,yi_flag,yi_time
46,20071.0,1.0,0.0,2017-03-17,NaT,0001-01-01,2017-03-17,0,0,0,0
132,20071.0,1.0,2.0,2022-10-24,NaT,0001-01-01,2022-10-24,1,31,1,0
5,20071.0,1.0,17.0,2022-10-24,NaT,0001-01-01,2022-10-24,0,0,0,0
306,854897.0,1.0,0.0,2010-01-01,NaT,0001-01-01,2010-01-01,0,0,0,0
500,854897.0,1.0,2.0,2024-05-17,NaT,0001-01-01,2024-05-17,1,31,1,0


In [16]:
df2_4['yi_flag'].value_counts()

yi_flag
0    52638
1    18172
Name: count, dtype: int64

In [17]:
#筛选出胰岛素都满足的数据
df2_4=pd.read_csv(os.path.join(datapath, 'df2_4.csv'))
# df2_5=df2_4[(df2_4['yi_flag']==1)&(df2_4['jia_flag']==1)]
# df2_5=df2_5.copy()
df2_5=df2_4.copy()
df2_5.drop(['jia_flag','yi_flag','jia_time','yi_time'],axis=1,inplace=True)
print(df2_5.shape)
df2_6=merge_period(df2_5)
print(df2_6.shape)

(70810, 7)
(70802, 7)


In [18]:
def compute_T1(df):
    df=df.copy()
    df['T0'] = pd.to_datetime(df['T0'], errors='coerce')
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
    df.loc[:, 'T1'] = pd.NaT
    df.loc[:,'opposite_date'] = pd.NaT
    df.loc[:,'yi_start_date'] = pd.NaT
    df['type_no_class'] = pd.to_numeric(df['type_no_class'], errors='coerce')

    for user_id, group in df.groupby('user_id'):
        # 遍历每个用户的行
        for index, row in group.iterrows():
            # 对于 type_no_class 为 1 或 2 的行，找到对比组药物的最早开始日期
            if row['type_no_class'] in [1, 2]:
                opposite_type = 1 if row['type_no_class'] == 2 else 2
                min_start_date = group[(group['type_no_class'] == opposite_type) &
                                       (group['start_date'] > row['start_date'])]['start_date'].min()
                if pd.notna(min_start_date):
                    df.at[index, 'opposite_date'] = min_start_date

                # 找到胰岛素的最早开始日期
                min_insulin_start = group[(group['type_no_class'].between(9, 19)) &
                                          (group['start_date'] > row['start_date'])]['start_date'].min()
                if pd.notna(min_insulin_start):
                    df.at[index, 'yi_start_date'] = min_insulin_start
                dates=[row['end_date'],min_start_date,min_insulin_start]
                
                valid_dates = [date for date in dates if pd.notna(date)]
                if valid_dates:
                    df.at[index, 'T1'] = min(valid_dates)
                else:
                    df.at[index, 'T1'] = pd.Timestamp('2023-09-01')
    return df


df2_7=compute_T1(df2_6)
print(df2_7.shape)
# df2_8=merge_period(df2_7)
# print(df2_8.shape)
# #时间提取完毕，删掉无用列
# df2_8=df2_8.copy()
# print(df2_8.shape)
df2_8=df2_7[df2_7['type_no_class'].isin([1,2])]
print(df2_8.shape)
df2_8.columns

(70802, 10)
(22080, 10)


Index(['user_id', 'drug_type', 'type_no_class', 'start_date', 'end_date',
       'adjust_date', 'T0', 'T1', 'opposite_date', 'yi_start_date'],
      dtype='object')

In [19]:
df2_8.drop(['start_date','end_date','opposite_date','yi_start_date'],axis=1,inplace=True)
df2_8.to_csv(os.path.join(datapath, 'df2_8.csv'), index=False)

C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\1654885613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_8.drop(['start_date','end_date','opposite_date','yi_start_date'],axis=1,inplace=True)


In [20]:
df2_8.head()

,user_id,drug_type,type_no_class,adjust_date,T0,T1
1,20071.0,1.0,2.0,0001-01-01,2022-10-24,2023-09-01
4,854897.0,1.0,2.0,0001-01-01,2024-05-17,2023-09-01
5,902487.0,1.0,2.0,0001-01-01,2021-08-23,2023-09-01
10,1133103.0,1.0,1.0,0001-01-01,2024-06-21,2023-09-01
11,1133103.0,1.0,2.0,0001-01-01,2020-07-01,2024-06-14


In [6]:
df2_8=pd.read_csv(os.path.join(datapath, 'df2_8.csv'))
count_user_id(df2_8)
df2_8['type_no_class'].value_counts()

user_id: 16165
5763
14677


type_no_class
2.0    15358
1.0     6722
Name: count, dtype: int64

In [7]:
def get_first_HbA1c(df1, df):
    df=df.copy()
    df['T0'] = pd.to_datetime(df['T0'], errors='coerce')
    df1.loc[:, 'visit_at'] = pd.to_datetime(df1['visit_at'], errors='coerce')
    df.loc[:, 'first_Hb_flag'] = 0  # 是否有基线访视
    df.loc[:, 'first_Hb_time'] = 0  # 基线访视距离T0的天数
    df.loc[:, 'first_Hb_date'] = 0  # 基线访视日期

    last_13_columns = df1.columns[-15:]#增加了hosp_id和age
    def calculate_and_update_group(group):
        user_visits = df1[df1['user_id'] == group.name]
        for index, row in group.iterrows():
            # 筛选出T0之前的访问
            valid_visits = user_visits[user_visits['visit_at'] <= row['T0']]
            if not valid_visits.empty:
                # 找到最近的访问
                latest_visit = valid_visits['visit_at'].max()
                days_difference = (row['T0'] - latest_visit).days
                group.at[index, 'first_Hb_time'] = days_difference
                group.at[index, 'first_Hb_date'] = latest_visit.date()#只需保存first_Hb_date的年月日
                # 根据 days_difference 更新 first_Hb_flag 和基线访视数据
                if days_difference < 92:
                    group.at[index, 'first_Hb_flag'] = 1
                    baseline_visit = valid_visits.sort_values('visit_at', ascending=False).iloc[0]
                    for col in last_13_columns:
                        group.at[index, col + '_base'] = baseline_visit[col]
            else:
                # 没有有效访问，保持 first_Hb_time 为 NaN
                group.at[index, 'first_Hb_time'] = pd.NA

        return group

    df2 = df.groupby('user_id').apply(calculate_and_update_group)
    df2.reset_index(drop=True, inplace=True)
    return df2
# df_v=pd.read_csv(os.path.join(datapath, 'df_val_visit.csv'))

#验证数据列信息调整
df_v.drop(['sex','eGFR_EPI'],axis=1,inplace=True)
df2_first=get_first_HbA1c(df_v, df2_8)
print(df2_first.shape)
print(df2_first.columns)

C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1744\482938588.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v.drop(['sex','eGFR_EPI'],axis=1,inplace=True)


(22080, 24)
Index(['user_id', 'drug_type', 'type_no_class', 'adjust_date', 'T0', 'T1',
       'first_Hb_flag', 'first_Hb_time', 'first_Hb_date', 'user_id_base',
       'visit_level_base', 'visit_at_base', 'hosp_id_base', 'age_base',
       'diabetic_duration_base', 'sbp_base', 'dbp_base', 'bmi_v1_base',
       'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base', 'CP0_base',
       'HbA1c_base'],
      dtype='object')


In [4]:
#临时统计，添加两列
def get_first_HbA1c(df1, df):
    df=df.copy()
    df['T0'] = pd.to_datetime(df['T0'], errors='coerce')
    df1.loc[:, 'visit_at'] = pd.to_datetime(df1['visit_at'], errors='coerce')
    df.loc[:, 'first_Hb_flag'] = 0  # 是否有基线访视
    df.loc[:, 'first_Hb_time'] = 0  # 基线访视距离T0的天数
    df.loc[:, 'first_Hb_date'] = 0  # 基线访视日期

    last_13_columns = df1.columns[-16:]#增加了hosp_id和age、sex
    def calculate_and_update_group(group):
        user_visits = df1[df1['user_id'] == group.name]
        for index, row in group.iterrows():
            # 筛选出T0之前的访问
            valid_visits = user_visits[user_visits['visit_at'] <= row['T0']]
            if not valid_visits.empty:
                # 找到最近的访问
                latest_visit = valid_visits['visit_at'].max()
                days_difference = (row['T0'] - latest_visit).days
                group.at[index, 'first_Hb_time'] = days_difference
                group.at[index, 'first_Hb_date'] = latest_visit.date()#只需保存first_Hb_date的年月日
                # 根据 days_difference 更新 first_Hb_flag 和基线访视数据
                if days_difference < 92:
                    group.at[index, 'first_Hb_flag'] = 1
                    baseline_visit = valid_visits.sort_values('visit_at', ascending=False).iloc[0]
                    for col in last_13_columns:
                        group.at[index, col + '_base'] = baseline_visit[col]
            else:
                # 没有有效访问，保持 first_Hb_time 为 NaN
                group.at[index, 'first_Hb_time'] = pd.NA

        return group

    df2 = df.groupby('user_id').apply(calculate_and_update_group)
    df2.reset_index(drop=True, inplace=True)
    return df2
df2_8=pd.read_csv(os.path.join(datapath, 'df2_8.csv'))
# df1_merged_statistics.columns
df2_first_statistics=get_first_HbA1c(df_v, df2_8)
df2_first_statistics.to_csv(os.path.join(datapath, 'df2_first_statistics.csv'), index=False)
df2_first_statistics.head()

,user_id,drug_type,type_no_class,adjust_date,T0,T1,first_Hb_flag,first_Hb_time,first_Hb_date,visit_level_base,...,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,sex_base,eGFR_EPI_base
0,20071.0,1.0,2.0,0001-01-01,2022-10-24,2023-09-01,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,854897.0,1.0,2.0,0001-01-01,2024-05-17,2023-09-01,0,605.0,2022-09-20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,902487.0,1.0,2.0,0001-01-01,2021-08-23,2023-09-01,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1133103.0,1.0,1.0,0001-01-01,2024-06-21,2023-09-01,1,39.0,2024-05-13,32.0,...,78.0,24.9,NaN,NaN,NaN,8.8,1.933,11.6,2.0,NaN
4,1133103.0,1.0,2.0,0001-01-01,2020-07-01,2024-06-14,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# 仅保留first_Hb_flag=1的行
def cut_hosp(df):
    df_valid=df.copy()
    df_valid= df_valid[(df_valid['hosp_id_base'] == 11362) | (df_valid['hosp_id_base'] == 10695) | (df_valid['hosp_id_base'] == 13619) | (df_valid['hosp_id_base'] == 12105)]
    return df_valid
df2_first=cut_hosp(df2_first)
count_user_id(df2_first)
df2_first1 = df2_first[(df2_first['first_Hb_flag'] == 1) ] 
print(df2_first1.shape)
df2_first2 = df2_first1[(df2_first1['age_base'] > 18) ] 
print(df2_first2.shape)
df2_first2 = df2_first2.copy()
df2_first2.drop(['first_Hb_flag', 'first_Hb_time'], axis=1, inplace=True)
count_user_id(df2_first2)
#仅保留基线糖化不正常的患者
df2_9 = df2_first2[df2_first2['HbA1c_base'] >= 7]
count_user_id(df2_9)
print(df2_9.shape)

user_id: 5645
2728
4534
(7979, 24)
(7978, 24)
user_id: 5645
2727
4534
user_id: 3988
1814
3165
(5253, 22)


In [42]:
def get_potential_last_HbA1c(df1, df):
    df=df.copy()
    df1['visit_at'] = pd.to_datetime(df1['visit_at'], errors='coerce')
    df['T1'] = pd.to_datetime(df['T1'], errors='coerce')
    df['T0'] = pd.to_datetime(df['T0'], errors='coerce')
    df.loc[:, 'last_Hb_date'] = 0  # 末次访视日期
    df.loc[:, 'last_Hb_empty'] = 0  # 是否找不到访视

    def calculate_and_update_group(group):
        user_visits = df1[df1['user_id'] == group.name]
        for index, row in group.iterrows():
            three_months_after_T1 = row['T1'] + pd.Timedelta(days=92)
            three_months_after_T0 = row['T0'] + pd.Timedelta(days=92)

            valid_visits = user_visits[(user_visits['visit_at'] >= three_months_after_T0) & (
                            user_visits['visit_at'] <= three_months_after_T1)]

            if not valid_visits.empty:
                # 找到可行范围内最远的访视
                last_visit_date = valid_visits['visit_at'].max()
                group.at[index, 'last_Hb_date'] = last_visit_date.date()  # 只需保存last_Hb_date的年月日
            else:
                group.at[index, 'last_Hb_empty'] = 1
        return group

    df2 = df.groupby('user_id').apply(calculate_and_update_group)
    #删除index
    df2.reset_index(drop=True, inplace=True)
    return df2

df3_0=get_potential_last_HbA1c(df_v, df2_9)
print(df3_0.shape)
print(df3_0.columns)
df3_0.to_csv(os.path.join(datapath, 'df3_0.csv'), index=False)

(9076, 24)
Index(['user_id', 'drug_type', 'type_no_class', 'adjust_date', 'T0', 'T1',
       'first_Hb_date', 'user_id_base', 'visit_level_base', 'visit_at_base',
       'hosp_id_base', 'age_base', 'diabetic_duration_base', 'sbp_base',
       'dbp_base', 'bmi_v1_base', 'ALT_base', 'LDL_c_base', 'Cr_base',
       'FPG_base', 'CP0_base', 'HbA1c_base', 'last_Hb_date', 'last_Hb_empty'],
      dtype='object')


In [65]:
# 仅保留last_Hb_empty=0的行
df3_0=pd.read_csv(os.path.join(datapath, 'df3_0.csv'))
df3_1 = df3_0[(df3_0['last_Hb_empty'] == 0) ] 
df3_1=cut_hosp(df3_1)
count_user_id(df3_1)
df3_1  = df3_1 .copy()
df3_1 .drop(['last_Hb_empty','sbp_base'], axis=1, inplace=True)
print(df3_1.shape)

user_id: 1791
635
1391
(2098, 22)


In [45]:
from collections import defaultdict
def count_overlaps(df):
    '''
    根据T0、T1统计出每个患者所有服用type_no_class=1和2的时间段
    计算同时服用的时间段和总天数
    如果有多段时间同时服用，则both_date是多个时间段
    如果没有同时服用的情况，both_date和both_time都为0
    '''
    filtered_data = df[df['type_no_class'].isin([1,2])]
    filtered_data.loc[:,'T0'] = pd.to_datetime(filtered_data.loc[:,'T0'], errors='coerce')
    filtered_data.loc[:,'T1'] = pd.to_datetime(filtered_data.loc[:,'T1'], errors='coerce')


    # 建立用药时间段字典
    medication_periods = defaultdict(lambda: {'class_1': [], 'class_2': []})
    for _, row in filtered_data.iterrows():
        user_id = row['user_id']
        class_type = 'class_1' if row['type_no_class'] == 1 else 'class_2'
        end_date = row['T1'] #if pd.notna(row['T1']) else pd.Timestamp('2023-09-01')
        medication_periods[user_id][class_type].append((row['T0'], end_date))

    # 计算重叠时间
    def find_overlaps_open_interval(periods_1, periods_2):
        overlaps = []
        total_days = 0
        for start_1, end_1 in periods_1:
            for start_2, end_2 in periods_2:
                start_overlap = max(start_1, start_2)
                end_overlap = min(end_1 if end_1 else start_overlap, end_2 if end_2 else start_overlap)
                if start_overlap < end_overlap:
                    overlap_days = (end_overlap - start_overlap).days
                    if overlap_days > 0:
                        # print(start_1,end_1, start_2, end_2, start_overlap, end_overlap, overlap_days)
                        overlaps.append((start_overlap, end_overlap))
                        total_days += overlap_days
        return overlaps, total_days

    results = defaultdict(lambda: {'both_date': [], 'both_time': 0})
    for user_id, periods in medication_periods.items():
        overlaps, total_days = find_overlaps_open_interval(periods['class_1'], periods['class_2'])
        results[user_id]['both_date'] = overlaps
        results[user_id]['both_time'] = total_days

    # 结果汇总
    results_df_open_interval = pd.DataFrame.from_dict(results, orient='index').reset_index()
    results_df_open_interval.rename(columns={'index': 'user_id'}, inplace=True)
    #统计同时服用的患者数量和患者总数
    both_time_count = (results_df_open_interval['both_time']>0).sum()
    print('同时服用的患者数量:',both_time_count)
    print('患者总数:',results_df_open_interval.shape[0])
    
    df_overlap=results_df_open_interval#.reset_index(drop=True, inplace=True)
    df_overlap=df_overlap[df_overlap['both_time']>0]
    print(df_overlap)
    return df_overlap
df_overlap=count_overlaps(df3_1)

同时服用的患者数量: 295
患者总数: 2902
        user_id                                     both_date  both_time
19    1375024.0  [(2022-12-27 00:00:00, 2023-09-01 00:00:00)]        248
51    1395880.0  [(2023-05-09 00:00:00, 2024-06-24 00:00:00)]        412
60    1399313.0  [(2023-06-02 00:00:00, 2023-09-01 00:00:00)]         91
62    1399483.0  [(2022-09-19 00:00:00, 2023-09-01 00:00:00)]        347
66    1401277.0  [(2021-11-15 00:00:00, 2023-09-19 00:00:00)]        673
...         ...                                           ...        ...
2830  5153114.0  [(2023-08-19 00:00:00, 2023-11-29 00:00:00)]        102
2831  5159085.0  [(2023-08-18 00:00:00, 2023-12-19 00:00:00)]        123
2834  5168598.0  [(2023-08-25 00:00:00, 2023-09-01 00:00:00)]          7
2838  5182779.0  [(2023-09-01 00:00:00, 2024-04-12 00:00:00)]        224
2860  5301799.0  [(2023-11-12 00:00:00, 2024-01-01 00:00:00)]         50

[295 rows x 3 columns]


In [46]:
#去掉重复组患者
print(df3_1.shape)
df3_11 = df3_1[~df3_1['user_id'].isin(df_overlap['user_id'])]
print(df3_11.shape)
df3_11.columns

(3360, 22)
(2755, 22)


Index(['user_id', 'drug_type', 'type_no_class', 'adjust_date', 'T0', 'T1',
       'first_Hb_date', 'user_id_base', 'visit_level_base', 'visit_at_base',
       'hosp_id_base', 'age_base', 'diabetic_duration_base', 'dbp_base',
       'bmi_v1_base', 'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base',
       'CP0_base', 'HbA1c_base', 'last_Hb_date'],
      dtype='object')

In [47]:
df3_12=df3_11.copy()
df3_12.drop([ 'drug_type','adjust_date'],axis=1,inplace=True)
df3_12.columns

Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date', 'user_id_base',
       'visit_level_base', 'visit_at_base', 'hosp_id_base', 'age_base',
       'diabetic_duration_base', 'dbp_base', 'bmi_v1_base', 'ALT_base',
       'LDL_c_base', 'Cr_base', 'FPG_base', 'CP0_base', 'HbA1c_base',
       'last_Hb_date'],
      dtype='object')

In [48]:
df3_12.to_csv(os.path.join(datapath, 'df3_12.csv'), index=False)
df3_12.head()

,user_id,type_no_class,T0,T1,first_Hb_date,user_id_base,visit_level_base,visit_at_base,hosp_id_base,age_base,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,last_Hb_date
1,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,1211166.0,6.0,2018-05-18,11362.0,42.0,60.0,59.0,24.7,10.0,2.79,31.9,10.82,0.868,9.4,2022-12-05
4,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,1240517.0,14.0,2020-04-16,11362.0,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,2021-04-06
5,1240517.0,2.0,2020-07-03,2023-01-11,2020-04-16,1240517.0,14.0,2020-04-16,11362.0,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,2023-04-11
7,1243003.0,2.0,2020-07-09,2023-09-01,2020-05-14,1243003.0,14.0,2020-05-14,11362.0,59.0,128.0,80.0,23.6,16.2,2.92,84.9,8.63,1.408,7.0,2023-04-17
11,1264655.0,1.0,2017-06-15,2020-03-25,2017-06-15,1264655.0,1.0,2017-06-15,11362.0,63.0,97.0,86.0,33.4,10.9,2.11,86.1,4.82,2.213,7.1,2020-06-03


In [18]:
#计算其他四种降糖药物联用情况
# other_label_type='series'#计算连续标记
other_label_type='discrete'#计算离散标记

In [14]:
def drug_type_other(df2):
    df2['start_date'] = pd.to_datetime(df2['start_date'], errors='coerce')
    df2_1  = df2[df2['flag_drug'] != 1]
    # 将 'type_no_class' 列转换为数值型
    df2_1.loc[:, 'type_no_class'] = pd.to_numeric(df2_1['type_no_class'], errors='coerce')

    # 将 'type_no_class' 列中的空值转换为0
    df2_1.loc[:, 'type_no_class'] = df2_1['type_no_class'].fillna(1).astype('Int64')

    df2_1.loc[df2_1['drug_no'].isin([13001]), 'type_no_class'] = 101
    df2_1.loc[df2_1['drug_no'].isin([13002]), 'type_no_class'] = 102
    df2_1.loc[df2_1['drug_no'].isin([13003]), 'type_no_class'] = 103
    df2_1.loc[df2_1['drug_no'].isin([13004]), 'type_no_class'] = 104
    df2_1.loc[df2_1['drug_no'].isin([13005]), 'type_no_class'] = 105
    df2_1.loc[df2_1['drug_no'].isin([13006]), 'type_no_class'] = 106
    df2_1.loc[df2_1['drug_no'].isin([13007]), 'type_no_class'] = 107
    df2_1.loc[df2_1['drug_no'].isin([67002]), 'type_no_class'] = 108
    df2_1.loc[df2_1['drug_no'].isin([17001]), 'type_no_class'] = 109
    df2_1.loc[df2_1['drug_no'].isin([17002]), 'type_no_class'] = 110
    df2_1.loc[df2_1['drug_no'].isin([17003]), 'type_no_class'] = 111
    
    df2_1.loc[df2_1['drug_no'].isin([14001]), 'type_no_class'] = 201
    df2_1.loc[df2_1['drug_no'].isin([14002]), 'type_no_class'] = 202
    
    df2_1.loc[df2_1['drug_no'].isin([15001]), 'type_no_class'] = 301
    df2_1.loc[df2_1['drug_no'].isin([15002]), 'type_no_class'] = 302
    df2_1.loc[df2_1['drug_no'].isin([15003]), 'type_no_class'] = 303
    
    df2_1.loc[df2_1['drug_no'].isin([16001]), 'type_no_class'] = 401
    df2_1.loc[df2_1['drug_no'].isin([16002]), 'type_no_class'] = 402

    df2_1.loc[:, 'type_no_class'] = df2_1['type_no_class'].replace({6: -1, 8: -2, 2: 0})

    df2_2 = df2_1[df2_1['type_no_class'].isin([-1, -2, 0,101,102,103,104,105,106,107,108,109,110,111,201,202,301,302,303,401,402])]

    df2_2 = df2_2.drop([ 'drug_no', 'flag_drug'], axis=1)

    df2_2.loc[:, 'type_no_class'] = df2_2['type_no_class'].replace({-1: 1, -2: 2})
    return df2_2

def adjust_drug_type_other(df):
    '''
    删除所有drug_type=2的行
    将所有drug_type=4的行的drug_type改为1
    对于所有drug_type=1的行，如果存在另一行满足user_id、start_date、type_no_class相同且drug_type=3，则删除这个drug_type=1的行
    '''
    df = df[df['drug_type'] != 2]
    df.loc[df['drug_type'] == 4, 'drug_type'] = 1
     # 初始化一个空的索引列表用于记录需要删除的行
    rows_to_remove = []
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
    # 遍历数据帧，检查每一对相邻的行
    for i in range(len(df) - 1):
        current_row = df.iloc[i]
        next_row = df.iloc[i + 1]

        # 如果当前行和下一行的user_id,type_no_class相同
        if (current_row['user_id'] == next_row['user_id']  and
                current_row['type_no_class'] == next_row['type_no_class'] ):
            # 并且当前行是drug_type=1，下一行是drug_type=3，那么删除当前行
            if (current_row['drug_type'] == 1 and next_row['drug_type'] == 3 and
                current_row['start_date'] == next_row['start_date']):
                rows_to_remove.append(i)
            

    # 删除需要删除的行
    df = df.drop(df.index[rows_to_remove])

    return df

def merge_medication_periods(df,lst):
    '''
    将type_no_class在lst中的所有时间段，按user_id分组合并
    '''
    df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
    df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
    # 初始化收集合并后时间段的列表
    merged_periods = []

    # 按user_id分组
    for user_id, group in df.groupby('user_id'):

        insulin_usage = group[group['type_no_class'].isin(lst)]

        # 如果没有对应的药物记录，继续下一个分组
        if insulin_usage.empty:
            continue

        # 排序，以便合并时间段
        insulin_usage = insulin_usage.sort_values(by='start_date')

        # 初始化第一个时间段
        current_start = None
        current_end = None
        for _, row in insulin_usage.iterrows():
            # 如果是第一个时间段，或者当前记录的开始时间在当前时间段之后
            if current_start is None or row['start_date'] > current_end:
                # 如果当前时间段已存在，保存当前时间段
                if current_start is not None:
                    merged_periods.append([user_id, current_start, current_end])
                # 重置当前时间段为新记录的时间段
                current_start = row['start_date']
                current_end = row['end_date']
            else:
                # 如果当前记录与当前时间段重叠，扩展当前时间段的结束时间
                current_end = max(current_end, row['end_date'])

        # 保存最后一个时间段
        if current_start is not None:
            merged_periods.append([user_id, current_start, current_end])

    # 创建新的DataFrame
    df2 = pd.DataFrame(merged_periods, columns=['user_id', 'start_date', 'end_date'])
    return df2

def check_overlap_with_df2_regression(df, df2,drug='other_flag',flag='series'):
    df=df.copy()
    df.loc[:, drug] = 0 
    df['first_Hb_date'] = pd.to_datetime(df['first_Hb_date'], errors='coerce')
    df['target_Hb_date'] = pd.to_datetime(df['target_Hb_date'], errors='coerce')
    df['last_Hb_date'] = pd.to_datetime(df['last_Hb_date'], errors='coerce')
    df2['start_date'] = pd.to_datetime(df2['start_date'], errors='coerce')
    df2['end_date'] = pd.to_datetime(df2['end_date'], errors='coerce')
    # 遍历df
    for index, row in df.iterrows():
        user_id = row['user_id']
        first_Hb_date = row['first_Hb_date']
        #如果row['target_Hb_date']不为空，则将last_Hb_date设置为row['target_Hb_date']，否则不变
        if pd.notna(row['target_Hb_date']):
            last_Hb_date = row['target_Hb_date']
        else:
            last_Hb_date = row['last_Hb_date']

        # 筛选出df2中相同user_id的所有记录
        user_records = df2[df2['user_id'] == user_id]

        # 检查是否有重叠
        for _, record in user_records.iterrows():
            start_date = record['start_date']
            end_date = record['end_date']

            # 检查时间段是否重叠
            if not (last_Hb_date < start_date or first_Hb_date > end_date):
                #如果flag是离散的
                if flag=='discrete':
                    df.at[index, drug] = 1
                #如果flag是连续的
                else:
                    #计算重叠时长相对于比例
                    overlap_start = max(first_Hb_date, start_date)
                    overlap_end = min(last_Hb_date, end_date)
                    df.at[index, drug] = (overlap_end - overlap_start).days / (last_Hb_date - first_Hb_date).days
    return df



In [51]:
df_other_1 = drug_type_other(df_d)
df_other_1.sort_values(by=['user_id',  'type_no_class','start_date','drug_type'], inplace=True)
df_other_2=adjust_drug_type_other(df_other_1)
df_other_2 = df_other_2.drop(['drug_type','adjust_date'], axis=1)
df_other_3=merge_period(df_other_2)
df_other_3.to_csv(os.path.join(datapath, 'df_other_3.csv'), index=False)

C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\2576849861.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\2576849861.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')
C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\2576849861.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [16]:
df_other_3=pd.read_csv(os.path.join(datapath, 'df_other_3.csv'))
df_v=pd.read_csv(os.path.join(datapath, 'df_val_visit.csv'))
df_v.drop(['sex','eGFR_EPI'],axis=1,inplace=True)
df_other_cumi=merge_medication_periods(df_other_3,[101,102,103,104,105,106,107,108,109,110,111])
df_other_jia=merge_medication_periods(df_other_3,[201,202])
df_other_tang=merge_medication_periods(df_other_3,[301,302,303])
df_other_TZD=merge_medication_periods(df_other_3,[401,402])

In [53]:
def get_all_normal_HbA1c(df1, df):
    df=df.copy()
    # 转换日期列为datetime类型
    df['T0'] = pd.to_datetime(df['T0'], errors='coerce')
    df['first_Hb_date'] = pd.to_datetime(df['first_Hb_date'], errors='coerce')
    df['last_Hb_date'] = pd.to_datetime(df['last_Hb_date'], errors='coerce')
    df1['visit_at'] = pd.to_datetime(df1['visit_at'], errors='coerce')

    df.loc[:, 'normal_Hb_flag'] = 0  # 是否有达标访视
    df.loc[:, 'normal_Hb_date'] = pd.NA  # 达标访视日期
    df.loc[:, 'drug_valid_time'] = 0  # 药物达标总天数时间
    last_columns = df1.columns[-1:]

    expanded_rows = []

    for _, row in df.iterrows():
        user_visits = df1[df1['user_id'] == row['user_id']]
        three_months_after_first = row['first_Hb_date'] + pd.Timedelta(days=92)
        twelve_months_after_first = row['first_Hb_date'] + pd.Timedelta(days=457)
        # 筛选出在指定日期范围内的访问记录
        valid_visits = user_visits[(user_visits['visit_at'] >= three_months_after_first)  
                                   & (user_visits['visit_at'] <= twelve_months_after_first)
                                   & (user_visits['visit_at'] <= row['last_Hb_date'])]
        # 筛选达标的访问记录
        normal_visits = valid_visits[valid_visits['HbA1c'] < 7].sort_values(by='visit_at')
        first_normal_visit_date = normal_visits.iloc[0]['visit_at'] if not normal_visits.empty else None

        if valid_visits.empty:
            # 如果没有找到任何访视，仍需复制原行但不添加访视信息
            row['normal_Hb_flag'] = 0
            expanded_rows.append(row)
        else:
            for _, visit in valid_visits.iterrows():
                new_row = row.copy()
                if first_normal_visit_date and visit['visit_at'] >= first_normal_visit_date:
                    # 达标或达标后的访视
                    new_row['normal_Hb_flag'] = 1
                else:
                    # 未达标或达标前的访视
                    new_row['normal_Hb_flag'] = 0
                new_row['drug_valid_time'] = (visit['visit_at'] - row['T0']).days
                new_row['normal_Hb_date'] = visit['visit_at'].date()

                for col in last_columns:
                    new_row[col + '_normal'] = visit[col]

                expanded_rows.append(new_row)

        # 转换expanded_rows列表为DataFrame
    expanded_df = pd.DataFrame(expanded_rows)

    return expanded_df.reset_index(drop=True)

In [56]:
df3_12=pd.read_csv(os.path.join(datapath, 'df3_12.csv'))
# df3_2=get_normal_HbAlc(df1_merged,df3_12)
df3_2=get_all_normal_HbA1c(df_v,df3_12)
df3_2 = df3_2.drop(['visit_at_base','user_id_base','visit_level_base'], axis=1)
df3_2.sort_values(by=['user_id', 'type_no_class', 'first_Hb_date'])
print(df3_2.shape)
print(df3_2.columns)
df3_2['normal_Hb_flag'].value_counts()

(4009, 21)
Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date', 'hosp_id_base',
       'age_base', 'diabetic_duration_base', 'dbp_base', 'bmi_v1_base',
       'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base', 'CP0_base',
       'HbA1c_base', 'last_Hb_date', 'normal_Hb_flag', 'normal_Hb_date',
       'drug_valid_time', 'HbA1c_normal'],
      dtype='object')


normal_Hb_flag
1    2216
0    1793
Name: count, dtype: int64

In [57]:
#调整列顺序
original_indices = list(range(len(df3_2.columns)))
original_indices.remove(16)
original_indices.remove(18)
original_indices.remove(19)
original_indices.remove(20)
new_indices = original_indices[:5]+ [18] + [16]+[5]+[19]+ original_indices[6:17]
df3_3 = df3_2.iloc[:, new_indices]
df3_3.rename(columns={df3_3.columns[5]: 'target_Hb_date'}, inplace=True)
print(df3_3.columns)
print(df3_3.shape)

Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date',
       'target_Hb_date', 'last_Hb_date', 'hosp_id_base', 'drug_valid_time',
       'age_base', 'diabetic_duration_base', 'dbp_base', 'bmi_v1_base',
       'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base', 'CP0_base',
       'HbA1c_base', 'normal_Hb_flag'],
      dtype='object')
(4009, 20)


C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\3929545019.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_3.rename(columns={df3_3.columns[5]: 'target_Hb_date'}, inplace=True)


In [58]:
#删除用药时长小于90天的
df3_31=df3_3[df3_3['drug_valid_time'] >=90]
df3_31.drop_duplicates(keep='first',inplace=True)
print(df3_31.shape)
df3_3=df3_31.copy()

(3576, 20)


C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1600\3173577556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_31.drop_duplicates(keep='first',inplace=True)


In [59]:
df3_3.head()

,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,age_base,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,42.0,60.0,59.0,24.7,10.0,2.79,31.9,10.82,0.868,9.4,0
1,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2019-06-20,2022-12-05,11362.0,331,42.0,60.0,59.0,24.7,10.0,2.79,31.9,10.82,0.868,9.4,0
2,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,0
3,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2021-04-06,2021-04-06,11362.0,277,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,0
4,1240517.0,2.0,2020-07-03,2023-01-11,2020-04-16,2020-10-12,2023-04-11,11362.0,101,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,0


In [60]:
df3_3.to_csv(os.path.join(datapath, 'df_first_normal.csv'), index=False)
#注意，最后一列normal_Hb_flag是标记.索引6是医院编号，索引7-17是用药时长+10列特征

In [10]:
df3_3=pd.read_csv(os.path.join(datapath, 'df_first_normal.csv'))
count_user_id(df3_3)

user_id: 2364
679
1756


In [61]:
df3_3['type_no_class'].value_counts()

type_no_class
2.0    2573
1.0    1003
Name: count, dtype: int64

In [62]:
print('hosp_id:',df3_3['hosp_id_base'].nunique())
df3_3['hosp_id_base'].value_counts()

hosp_id: 31


hosp_id_base
11362.0    1442
11029.0     991
10695.0     653
12220.0      69
12865.0      67
12874.0      51
13619.0      50
58822.0      46
12105.0      41
12802.0      20
11868.0      17
35349.0      16
12519.0      15
13140.0      14
60529.0      14
31730.0       9
13505.0       9
30925.0       7
61242.0       6
12872.0       5
12024.0       5
60417.0       5
61222.0       4
61223.0       4
12106.0       3
12876.0       3
11208.0       3
12870.0       2
12873.0       2
60649.0       2
59584.0       1
Name: count, dtype: int64

In [63]:
df_first_normal=pd.read_csv(os.path.join(datapath, 'df_first_normal.csv'))
df_first_normal1 = check_overlap_with_df2_regression(df_first_normal, df_other_cumi,drug='cumi_flag',flag=other_label_type)
df_first_normal2= check_overlap_with_df2_regression(df_first_normal1, df_other_jia,drug='jia_flag',flag=other_label_type)
df_first_normal3 = check_overlap_with_df2_regression(df_first_normal2, df_other_tang,drug='tang_flag',flag=other_label_type)
df_first_normal4 = check_overlap_with_df2_regression(df_first_normal3, df_other_TZD,drug='TZD_flag',flag=other_label_type)

In [64]:
df_first_normal4.head()

,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,age_base,...,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag,cumi_flag,jia_flag,tang_flag,TZD_flag
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,42.0,...,2.79,31.9,10.82,0.868,9.4,0,1,1,1,0
1,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2019-06-20,2022-12-05,11362.0,331,42.0,...,2.79,31.9,10.82,0.868,9.4,0,1,1,1,0
2,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,67.0,...,2.01,116.8,8.06,3.541,7.7,0,1,1,0,0
3,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2021-04-06,2021-04-06,11362.0,277,67.0,...,2.01,116.8,8.06,3.541,7.7,0,1,1,0,0
4,1240517.0,2.0,2020-07-03,2023-01-11,2020-04-16,2020-10-12,2023-04-11,11362.0,101,67.0,...,2.01,116.8,8.06,3.541,7.7,0,1,1,0,0


In [65]:
df_first_normal4.shape

(3576, 24)

In [66]:
#调整列顺序
original_indices = list(range(len(df_first_normal4.columns)))
original_indices.remove(20)
original_indices.remove(21)
original_indices.remove(22)
original_indices.remove(23)
new_indices = original_indices[:9]+ [20] + [21]+[22]+[23]+ original_indices[9:]
df_first_normal5 = df_first_normal4.iloc[:, new_indices]
df_first_normal5.rename(columns={df_first_normal5.columns[5]: 'target_Hb_date'}, inplace=True)
print(df_first_normal5.columns)
print(df_first_normal5.shape)

Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date',
       'target_Hb_date', 'last_Hb_date', 'hosp_id_base', 'drug_valid_time',
       'cumi_flag', 'jia_flag', 'tang_flag', 'TZD_flag', 'age_base',
       'diabetic_duration_base', 'dbp_base', 'bmi_v1_base', 'ALT_base',
       'LDL_c_base', 'Cr_base', 'FPG_base', 'CP0_base', 'HbA1c_base',
       'normal_Hb_flag'],
      dtype='object')
(3576, 24)


In [67]:
df_first_normal5.drop_duplicates(subset=['user_id', 'type_no_class', 'first_Hb_date', 'target_Hb_date'],keep='first',inplace=True)
print(df_first_normal5.shape)

(3544, 24)


In [68]:
df_first_normal5.to_csv(os.path.join(datapath, 'df_first_normal_other.csv'), index=False)

In [2]:
def get_three_to_fifteen_month_HbA1c(df1, df,flag='orignal'):
    '''
    :param flag: orignal表示使用原始血糖数据，first_normal表示达标后使用第一次达标血糖数据
    '''
    df=df.copy()
    df['T0'] = pd.to_datetime(df['T0'], errors='coerce')
    df1['visit_at'] = pd.to_datetime(df1['visit_at'], errors='coerce')
    df['first_Hb_date'] = pd.to_datetime(df['first_Hb_date'], errors='coerce')
    df['last_Hb_date'] = pd.to_datetime(df['last_Hb_date'], errors='coerce')

    df.loc[:, 'xx_month_Hb_flag'] = 0  # 是否找不到访视(没有有3~15月的)
    df.loc[:, 'xx_month_Hb_date'] = None  # 目标访视日期
    df.loc[:, 'drug_regression_time'] = None  # 药物终点总天数
    last_columns = df1.columns[-1:]
    expanded_rows = []

    for _, row in df.iterrows():
        user_visits = df1[df1['user_id'] == row['user_id']]
        three_months_after_first = row['first_Hb_date'] + pd.Timedelta(days=90)
        twelve_months_after_first = row['first_Hb_date'] + pd.Timedelta(days=457)
        valid_visits = user_visits[(user_visits['visit_at'] >= three_months_after_first)
                                   & (user_visits['visit_at'] <= twelve_months_after_first)
                                   &( user_visits['visit_at'] <=row['last_Hb_date'])]
        if flag=='first_normal':
            # Identify the first normal visit (HbA1c < 7)
            normal_visits = valid_visits[valid_visits['HbA1c'] < 7].sort_values(by='visit_at')
            first_normal_hba1c = normal_visits.iloc[0]['HbA1c'] if not normal_visits.empty else None
            first_normal_visit_date = normal_visits.iloc[0]['visit_at'] if not normal_visits.empty else None

        if valid_visits.empty:
            row['xx_month_Hb_flag'] = 0
            expanded_rows.append(row)
        else:
            for _, visit in valid_visits.iterrows():
                new_row = row.copy()
                new_row['xx_month_Hb_flag'] = 1
                new_row['xx_month_Hb_date'] = visit['visit_at'].date()
                new_row['drug_regression_time'] = (visit['visit_at'] - row['T0']).days

                if flag=='first_normal':
                    # After the first normal visit, use its HbA1c value
                    if first_normal_visit_date and visit['visit_at'] >= first_normal_visit_date:
                        new_hba1c_value = first_normal_hba1c  # Set to first normal HbA1c value
                    else:
                        new_hba1c_value = visit['HbA1c']  # Use the actual HbA1c value before first normal visit

                    # Set the new HbA1c value for the visit and adjust other columns as needed
                    new_row['HbA1c'] = new_hba1c_value

                # 增加目标访视数据
                for col in last_columns:
                    new_row[col + '_xx_month'] = visit[col]
                expanded_rows.append(new_row)

    # 将expanded_rows列表转换为DataFrame
    expanded_df = pd.DataFrame(expanded_rows)

    return expanded_df.reset_index(drop=True)

In [58]:
#修改点1/3
# df3_4=get_three_to_fifteen_month_HbA1c(df_v,df3_3,flag='original')
df3_4=get_three_to_fifteen_month_HbA1c(df_v,df3_3,flag='first_normal')
print(df3_3.shape)
print(df3_4.shape)
print(df3_4.columns)
df3_4['xx_month_Hb_flag'].value_counts()#找不到访视的，上一轮已经踢掉了
df3_5 = df3_4[(df3_4['xx_month_Hb_flag'] == 1) ] 

(3576, 20)
(6518, 25)
Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date',
       'target_Hb_date', 'last_Hb_date', 'hosp_id_base', 'drug_valid_time',
       'age_base', 'diabetic_duration_base', 'dbp_base', 'bmi_v1_base',
       'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base', 'CP0_base',
       'HbA1c_base', 'normal_Hb_flag', 'xx_month_Hb_flag', 'xx_month_Hb_date',
       'drug_regression_time', 'HbA1c', 'HbA1c_xx_month'],
      dtype='object')


In [59]:
df3_5.drop_duplicates(subset=['user_id', 'type_no_class', 'first_Hb_date', 'xx_month_Hb_date'],keep='first',inplace=True)
print(df3_5.shape)
df3_5.head()

(3722, 25)


,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,age_base,...,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag,xx_month_Hb_flag,xx_month_Hb_date,drug_regression_time,HbA1c,HbA1c_xx_month
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,42.0,...,31.9,10.82,0.868,9.4,0,1,2018-12-12,141,7.9,7.9
1,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,42.0,...,31.9,10.82,0.868,9.4,0,1,2019-06-20,331,8.9,8.9
4,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,67.0,...,116.8,8.06,3.541,7.7,0,1,2020-10-12,101,7.7,7.7
5,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,67.0,...,116.8,8.06,3.541,7.7,0,1,2021-04-06,277,7.3,7.3
12,1243003.0,2.0,2020-07-09,2023-09-01,2020-05-14,2021-04-28,2023-04-17,11362.0,293,59.0,...,84.9,8.63,1.408,7.0,1,1,2021-04-28,293,6.7,6.7


In [60]:
#调整列顺序
original_indices = list(range(len(df3_5.columns)))
original_indices.remove(20)
original_indices.remove(21)
original_indices.remove(22)
new_indices = original_indices[:5]+[21]+[6] +[7]+[22]+ original_indices[9:19]+[23]
df3_6 = df3_5.iloc[:, new_indices]
df3_6.rename(columns={df3_6.columns[5]: 'target_Hb_date'}, inplace=True)
print(df3_6.columns)
print(df3_6.shape)


Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date',
       'target_Hb_date', 'last_Hb_date', 'hosp_id_base',
       'drug_regression_time', 'age_base', 'diabetic_duration_base',
       'dbp_base', 'bmi_v1_base', 'ALT_base', 'LDL_c_base', 'Cr_base',
       'FPG_base', 'CP0_base', 'HbA1c_base', 'HbA1c'],
      dtype='object')
(3722, 20)


C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_9528\2846984645.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_6.rename(columns={df3_6.columns[5]: 'target_Hb_date'}, inplace=True)


In [61]:
df3_6.head()

,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_regression_time,age_base,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,HbA1c
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,42.0,60.0,59.0,24.7,10.0,2.79,31.9,10.82,0.868,9.4,7.9
1,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2019-06-20,2022-12-05,11362.0,331,42.0,60.0,59.0,24.7,10.0,2.79,31.9,10.82,0.868,9.4,8.9
4,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,7.7
5,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2021-04-06,2021-04-06,11362.0,277,67.0,231.0,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,7.3
12,1243003.0,2.0,2020-07-09,2023-09-01,2020-05-14,2021-04-28,2023-04-17,11362.0,293,59.0,128.0,80.0,23.6,16.2,2.92,84.9,8.63,1.408,7.0,6.7


In [62]:
# df3_6.to_csv(os.path.join(datapath, 'df_six_month_Hb.csv', index=False)
#注意，最后一列HbA1c_six_month是回归标记.索引6是医院编号，索引7-17是用药时长+10列特征
df3_6.to_csv(os.path.join(datapath, 'df_xx_month_Hb.csv'), index=False)

In [63]:
df3_6['type_no_class'].value_counts()

type_no_class
2.0    2691
1.0    1031
Name: count, dtype: int64

In [64]:
print('hosp_id:',df3_6['hosp_id_base'].nunique())


hosp_id: 31


In [65]:
df_xx_month_Hb=pd.read_csv(os.path.join(datapath, 'df_xx_month_Hb.csv'))
df_xx_month_Hb1 = check_overlap_with_df2_regression(df_xx_month_Hb, df_other_cumi,drug='cumi_flag',flag=other_label_type)
df_xx_month_Hb2= check_overlap_with_df2_regression(df_xx_month_Hb1, df_other_jia,drug='jia_flag',flag=other_label_type)
df_xx_month_Hb3 = check_overlap_with_df2_regression(df_xx_month_Hb2, df_other_tang,drug='tang_flag',flag=other_label_type)
df_xx_month_Hb4 = check_overlap_with_df2_regression(df_xx_month_Hb3, df_other_TZD,drug='TZD_flag',flag=other_label_type)

In [66]:
df_xx_month_Hb4.head(10)

,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_regression_time,age_base,...,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,HbA1c,cumi_flag,jia_flag,tang_flag,TZD_flag
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,42.0,...,2.79,31.9,10.82,0.868,9.4,7.9,1,1,1,0
1,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2019-06-20,2022-12-05,11362.0,331,42.0,...,2.79,31.9,10.82,0.868,9.4,8.9,1,1,1,0
2,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,67.0,...,2.01,116.8,8.06,3.541,7.7,7.7,1,1,0,0
3,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2021-04-06,2021-04-06,11362.0,277,67.0,...,2.01,116.8,8.06,3.541,7.7,7.3,1,1,0,0
4,1243003.0,2.0,2020-07-09,2023-09-01,2020-05-14,2021-04-28,2023-04-17,11362.0,293,59.0,...,2.92,84.9,8.63,1.408,7.0,6.7,0,1,0,0
5,1264655.0,1.0,2017-06-15,2020-03-25,2017-06-15,2017-11-24,2020-06-03,11362.0,162,63.0,...,2.11,86.1,4.82,2.213,7.1,7.9,0,1,1,0
6,1264655.0,1.0,2017-06-15,2020-03-25,2017-06-15,2018-06-04,2020-06-03,11362.0,354,63.0,...,2.11,86.1,4.82,2.213,7.1,8.6,0,1,1,0
7,1264655.0,1.0,2020-06-03,2022-07-09,2020-06-03,2020-12-02,2022-04-29,11362.0,182,66.0,...,1.46,76.3,9.22,3.093,7.4,6.7,0,1,0,0
8,1264976.0,2.0,2020-09-04,2021-06-23,2020-09-04,2021-04-28,2021-04-28,11362.0,236,45.0,...,2.56,45.9,12.71,2.610,7.7,8.1,1,1,1,0
9,1308832.0,2.0,2022-08-09,2023-01-17,2022-08-05,2023-01-17,2023-01-17,11362.0,161,58.0,...,1.78,203.5,6.50,5.443,7.7,7.3,0,0,1,0


In [67]:
df_xx_month_Hb4.shape

(3722, 24)

In [68]:
#调整列顺序
original_indices = list(range(len(df_xx_month_Hb4.columns)))
original_indices.remove(20)
original_indices.remove(21)
original_indices.remove(22)
original_indices.remove(23)
new_indices = original_indices[:9]+ [20] + [21]+[22]+[23]+ original_indices[9:]
df_xx_month_Hb5 = df_xx_month_Hb4.iloc[:, new_indices]
df_xx_month_Hb5.rename(columns={df_xx_month_Hb5.columns[5]: 'target_Hb_date'}, inplace=True)
print(df_xx_month_Hb5.columns)
print(df_xx_month_Hb5.shape)

Index(['user_id', 'type_no_class', 'T0', 'T1', 'first_Hb_date',
       'target_Hb_date', 'last_Hb_date', 'hosp_id_base',
       'drug_regression_time', 'cumi_flag', 'jia_flag', 'tang_flag',
       'TZD_flag', 'age_base', 'diabetic_duration_base', 'dbp_base',
       'bmi_v1_base', 'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base',
       'CP0_base', 'HbA1c_base', 'HbA1c'],
      dtype='object')
(3722, 24)


In [69]:
df_xx_month_Hb5.to_csv(os.path.join(datapath, 'df_xx_month_Hb_other.csv'), index=False)

In [11]:
def fillna_with_imp(df):
    df_fill=df.copy()
    # 定义需要填补的列
    columns_to_impute = ['age_base', 'diabetic_duration_base', 'dbp_base',
       'bmi_v1_base', 'ALT_base', 'LDL_c_base', 'Cr_base', 'FPG_base',
       'CP0_base', 'HbA1c_base']

    # 创建多重填补器实例
    imp = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0, sample_posterior=True)

    # 只填补指定的列，保持索引
    df_imputed_sub = imp.fit_transform(df[columns_to_impute])
    df_imputed_sub = pd.DataFrame(df_imputed_sub, columns=columns_to_impute, index=df.index)  # 指定index参数
    
    # 将填补好的列合并回原始DataFrame
    df_fill[columns_to_impute] = df_imputed_sub
    return df_fill
def filter_max(df):
    df = df.copy()
    print("Initial shape:", df.shape)
    # filter_max_1 = df['dbp_base'] >= 40
    filter_max_2 = df['bmi_v1_base'] <= 60

    # 计算 ALT_base 的 0.1% 和 99.9% 百分位数
    lower_bound_ALT = df['ALT_base'].quantile(0.01)
    upper_bound_ALT = df['ALT_base'].quantile(0.99)
    df.loc[df['ALT_base'] < lower_bound_ALT, 'ALT_base'] = np.nan
    df.loc[df['ALT_base'] > upper_bound_ALT, 'ALT_base'] = np.nan
    print("bound of ALT:", lower_bound_ALT, upper_bound_ALT)

    # upper_bound_Cr = df['Cr_base'].quantile(0.99)
    # df.loc[df['Cr_base'] > upper_bound_Cr, 'Cr_base'] = np.nan
    # print("Upper bound of Cr:", upper_bound_Cr)
    df.loc[df['dbp_base'] < 40, 'dbp_base'] = np.nan
    
    filter_max_6 = df['FPG_base'] <= 40

    upper_bound_CP0 = df['CP0_base'].quantile(0.999)
    df.loc[df['CP0_base'] > upper_bound_CP0, 'CP0_base'] = np.nan
    print("Upper bound of CP0:", upper_bound_CP0)

    filter_max_8 = df['HbA1c_base'] <= 13.13

    filter_lst = [ filter_max_2,filter_max_8, filter_max_6]

    for i, filter_criteria in enumerate(filter_lst):
        df = df[filter_criteria]
        print( df.shape[0], end=' ')

    print()
    return df

In [67]:
#人数统计
df_first_normal_other=pd.read_csv(os.path.join(datapath, 'df_first_normal_other.csv'))
df_first_normal_other=cut_hosp(df_first_normal_other)
df_first_normal_other_delete=df_first_normal_other[df_first_normal_other['HbA1c_base']> 13.13].copy()
df_first_normal_other_copy=df_first_normal_other[df_first_normal_other['HbA1c_base']<= 13.13].copy()
count_user_id(df_first_normal_other_delete)
count_user_id(df_first_normal_other_copy)
df_first_normal_other_copy=fillna_with_imp(df_first_normal_other_copy)
df_first_normal_other_copy=filter_max(df_first_normal_other_copy)
df_first_normal_other_copy=fillna_with_imp(df_first_normal_other_copy)
count_user_id(df_first_normal_other_copy)
df_first_normal_other_copy['type_no_class'].value_counts()

user_id: 42
13
29
user_id: 1459
421
1088
Initial shape: (2097, 24)
bound of ALT: 7.892 158.4
Upper bound of CP0: 6.562488000000021
2097 2097 2097 
user_id: 1459
421
1088


type_no_class
2.0    1521
1.0     576
Name: count, dtype: int64

In [68]:
#按用药时长拆分
df_first_normal_other_updated=pd.read_csv(os.path.join(datapath, 'df_first_normal_other.csv'))
df_first_normal_other_updated=cut_hosp(df_first_normal_other)
#删除hosp_id效果不好的
df_first_normal_other_updated = df_first_normal_other_updated[~df_first_normal_other_updated['hosp_id_base'].isin([12220, 12874])]
# df_first_normal_other_updated.drop(df_first_normal_other_updated.columns[-1:], axis=1, inplace=True)
df_6_valid1=df_first_normal_other_updated[df_first_normal_other_updated['drug_valid_time']<=213] 
df_6_valid=df_6_valid1[df_6_valid1['drug_valid_time']<=213]
df_12_valid=df_first_normal_other_updated[df_first_normal_other_updated['drug_valid_time']>213]
print(df_6_valid['type_no_class'].value_counts())
print(df_12_valid['type_no_class'].value_counts())
#填充缺失值
# df_3_valid_fill=fillna_with_imp(df_3_valid)
df_6_valid_fill=fillna_with_imp(df_6_valid)
df_12_valid_fill=fillna_with_imp(df_12_valid)
#删除异常最大值
# df_3_valid_fill=filter_max(df_3_valid_fill)
df_6_valid_fill=filter_max(df_6_valid_fill)
df_12_valid_fill=filter_max(df_12_valid_fill)
#填充缺失值
# df_3_valid_fill=fillna_with_imp(df_3_valid_fill)
df_6_valid_fill=fillna_with_imp(df_6_valid_fill)
df_12_valid_fill=fillna_with_imp(df_12_valid_fill)
count_user_id(df_6_valid_fill)
count_user_id(df_12_valid_fill)
print(df_6_valid_fill['type_no_class'].value_counts())
print(df_12_valid_fill['type_no_class'].value_counts())
# df_3_valid_fill.to_csv(os.path.join(datapath, 'df_3_valid.csv'), index=False)
df_6_valid_fill.to_csv(os.path.join(datapath, 'df_6_valid.csv'), index=False)
df_12_valid_fill.to_csv(os.path.join(datapath, 'df_12_valid.csv'), index=False)
df_6_valid_fill.iloc[:,-16:].describe()

type_no_class
2.0    864
1.0    358
Name: count, dtype: int64
type_no_class
2.0    698
1.0    236
Name: count, dtype: int64
Initial shape: (1222, 24)
bound of ALT: 8.6104 153.87699999999975
Upper bound of CP0: 6.736822000000049
1222 1185 1185 
Initial shape: (934, 24)
bound of ALT: 6.9 157.34399999999988
Upper bound of CP0: 5.712955000000105
934 912 912 


C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1744\1484944402.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter_criteria]
C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_1744\1484944402.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter_criteria]


user_id: 1103
324
810
user_id: 817
206
624
type_no_class
2.0    836
1.0    349
Name: count, dtype: int64
type_no_class
2.0    685
1.0    227
Name: count, dtype: int64


,drug_valid_time,cumi_flag,jia_flag,tang_flag,TZD_flag,age_base,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag
count,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000,1185.000000
mean,144.037131,0.156962,0.788186,0.454008,0.059072,52.317300,79.218407,78.712150,26.516368,34.025166,3.026944,63.811288,8.935094,1.956496,9.003797,0.613502
std,30.756679,0.363919,0.408766,0.498090,0.235858,13.069742,91.749286,11.203503,4.087854,25.045496,1.085520,23.604714,2.806902,1.026666,1.570324,0.487152
min,90.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000,44.000000,13.749821,-32.711349,-0.508367,14.241023,3.000000,-0.417701,7.000000,0.000000
25%,121.000000,0.000000,1.000000,0.000000,0.000000,41.000000,1.000000,71.000000,23.700000,17.000000,2.260000,50.000000,6.910000,1.249000,7.700000,0.000000
50%,140.000000,0.000000,1.000000,0.000000,0.000000,53.000000,43.000000,78.000000,26.000000,25.600000,2.940000,61.300000,8.440000,1.796000,8.700000,1.000000
75%,167.000000,0.000000,1.000000,1.000000,0.000000,63.000000,131.000000,85.000000,28.600000,42.000000,3.690000,72.600000,10.460000,2.532000,10.100000,1.000000
max,212.000000,1.000000,1.000000,1.000000,1.000000,81.000000,551.000000,137.000000,45.000000,148.900000,10.040000,335.400000,20.000000,6.567000,13.100000,1.000000


In [73]:
#按用药时长拆分
df_xx_month_Hb_other_updated=pd.read_csv(os.path.join(datapath, 'df_xx_month_Hb_other.csv'))
# df_xx_month_Hb_other_updated.drop(df_xx_month_Hb_other_updated.columns[-1:], axis=1, inplace=True)

#删除hosp_id效果不好的
df_xx_month_Hb_other_updated = df_xx_month_Hb_other_updated[~df_xx_month_Hb_other_updated['hosp_id_base'].isin([12220, 12874])]
# df_3_regression=df_xx_month_Hb_other_updated[df_xx_month_Hb_other_updated['drug_regression_time']<=123]
df_6_regression1=df_xx_month_Hb_other_updated[df_xx_month_Hb_other_updated['drug_regression_time']<=213] 
df_6_regression=df_6_regression1[df_6_regression1['drug_regression_time']<=213]
df_12_regression=df_xx_month_Hb_other_updated[df_xx_month_Hb_other_updated['drug_regression_time']>213]
# df_3_regression_fill=fillna_with_imp(df_3_regression)
df_6_regression_fill=fillna_with_imp(df_6_regression)
df_12_regression_fill=fillna_with_imp(df_12_regression)

# df_3_regression_fill=filter_max(df_3_regression_fill)
df_6_regression_fill=filter_max(df_6_regression_fill)
df_12_regression_fill=filter_max(df_12_regression_fill)

# df_3_regression_fill=fillna_with_imp(df_3_regression_fill)
df_6_regression_fill=fillna_with_imp(df_6_regression_fill)
df_12_regression_fill=fillna_with_imp(df_12_regression_fill)

#修改点2/3
# df_3_regression_fill.to_csv(os.path.join(datapath, 'df_3_regression.csv'), index=False)
# df_6_regression_fill.to_csv(os.path.join(datapath, 'df_6_regression.csv'), index=False)
# df_12_regression_fill.to_csv(os.path.join(datapath, 'df_12_regression.csv'), index=False)

df_6_regression_fill.to_csv(os.path.join(datapath, 'df_6_regression_first_normal.csv'), index=False)
df_12_regression_fill.to_csv(os.path.join(datapath, 'df_12_regression_first_normal.csv'), index=False)

# print(df_3_regression_fill['type_no_class'].value_counts())
print(df_6_regression_fill['type_no_class'].value_counts())
print(df_12_regression_fill['type_no_class'].value_counts())
df_6_regression_fill.iloc[:,-16:].describe()

Initial shape: (2043, 24)
bound of ALT: 6.6921176551958546 148.958
Upper bound of CP0: 7.6440400000001185
2043 1969 1969 
Initial shape: (1552, 24)
bound of ALT: 6.9510000000000005 144.0
Upper bound of CP0: 6.856750000000051
1552 1497 1497 


C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_9528\1484944402.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter_criteria]
C:\Users\shijuan_125623147720\AppData\Local\Temp\1\ipykernel_9528\1484944402.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[filter_criteria]


type_no_class
2.0    1390
1.0     579
Name: count, dtype: int64
type_no_class
2.0    1100
1.0     397
Name: count, dtype: int64


,drug_regression_time,cumi_flag,jia_flag,tang_flag,TZD_flag,age_base,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,HbA1c
count,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000
mean,137.634332,0.212798,0.790249,0.406805,0.054850,52.254444,85.280405,79.847565,26.921168,33.231458,3.021802,65.330467,9.096761,2.036672,9.056181,6.957937
std,38.201905,0.409390,0.407234,0.491363,0.227745,12.863407,91.640650,11.365157,4.099619,24.821649,1.027855,22.513734,3.022631,1.102460,1.572873,1.228269
min,3.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000,44.000000,17.800000,-5.578543,0.330000,15.404300,3.000000,-1.334824,7.000000,4.300000
25%,110.000000,0.000000,1.000000,0.000000,0.000000,42.000000,1.000000,72.000000,24.200000,16.800000,2.300000,51.800000,7.080000,1.267000,7.700000,6.100000
50%,136.000000,0.000000,1.000000,0.000000,0.000000,54.000000,60.000000,79.000000,26.500000,24.800000,2.950000,62.900000,8.510000,1.874000,8.800000,6.790000
75%,167.000000,0.000000,1.000000,1.000000,0.000000,63.000000,139.000000,87.000000,28.900000,40.000000,3.640000,74.800000,10.560000,2.640000,10.130000,7.500000
max,212.000000,1.000000,1.000000,1.000000,1.000000,81.000000,551.000000,146.000000,50.600000,148.900000,10.040000,335.400000,31.910000,7.280000,13.100000,16.150000


In [69]:
#数据统计出表模块,缺失值填充后，按分类模型的来
df_6_valid=pd.read_csv(os.path.join(datapath, 'df_6_valid.csv'))
df_12_valid=pd.read_csv(os.path.join(datapath, 'df_12_valid.csv'))
df_6_regression=pd.read_csv(os.path.join(datapath, 'df_6_regression.csv'))
df_12_regression=pd.read_csv(os.path.join(datapath, 'df_12_regression.csv'))
df_valid = pd.concat([df_6_valid, df_12_valid], axis=0)
df_regression = pd.concat([df_6_regression, df_12_regression], axis=0)
print(df_6_valid.shape,df_12_valid.shape,df_valid.shape)
def cut_hosp(df):
    df_valid=df.copy()
    df_valid= df_valid[(df_valid['hosp_id_base'] == 11362) | (df_valid['hosp_id_base'] == 10695) | (df_valid['hosp_id_base'] == 13619) | (df_valid['hosp_id_base'] == 12105)]
    return df_valid
df_6_valid=cut_hosp(df_6_valid)
df_12_valid=cut_hosp(df_12_valid)
df_6_regression=cut_hosp(df_6_regression)
df_12_regression=cut_hosp(df_12_regression)
df_valid=cut_hosp(df_valid)
df_regression=cut_hosp(df_regression)
print(df_6_valid.shape,df_12_valid.shape,df_valid.shape)
print(df_6_regression.shape,df_12_regression.shape,df_regression.shape)
count_user_id(df_6_valid)
count_user_id(df_12_valid)
df_valid.head()

(1185, 24) (912, 24) (2097, 24)
(1185, 24) (912, 24) (2097, 24)
(1300, 24) (912, 24) (2212, 24)
user_id: 1103
324
810
user_id: 817
206
624


,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,cumi_flag,...,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2018-12-12,2022-12-05,11362.0,141,1,...,60.0,59.000000,24.70000,10.0,2.79,31.9,10.82,0.868,9.4,0
1,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2020-10-12,2021-04-06,11362.0,101,1,...,231.0,64.000000,21.10000,10.1,2.01,116.8,8.06,3.541,7.7,0
2,1264655.0,1.0,2017-06-15,2020-03-25,2017-06-15,2017-11-24,2020-06-03,11362.0,162,0,...,97.0,86.000000,33.40000,10.9,2.11,86.1,4.82,2.213,7.1,0
3,1264655.0,1.0,2020-06-03,2022-07-09,2020-06-03,2020-12-02,2022-04-29,11362.0,182,0,...,133.0,75.533748,30.18717,10.1,1.46,76.3,9.22,3.093,7.4,1
4,1308832.0,2.0,2022-08-09,2023-01-17,2022-08-05,2023-01-17,2023-01-17,11362.0,161,0,...,62.0,63.000000,27.20000,15.5,1.78,203.5,6.50,5.443,7.7,0


In [19]:
# #验证集合并医院
# def replace_values(D, K, b):
#     for df in D:
#         # 确保 A 列是字符串，转换为整数
#         df['hosp_id_base'] = df['hosp_id_base'].astype(int)

#         # 替换 A 列中在 K 中的值为 b
#         df['hosp_id_base'] = df['hosp_id_base'].replace(K, b)

#         # 验证替换是否完成
#         replaced_values = df['hosp_id_base'].isin(K).any()
#         if not replaced_values:
#             print("替换成功！没有未替换的值。")
#         else:
#             print("存在未替换的值！")
# print(df_6_regression.value_counts('hosp_id_base'))

# # replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [10695,58822,30925,13619,12105], 11362)
# replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [10695,13619,12105], 11362)
# # replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [10695,58822,30925,13619], 11362)
# # replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [10695,58822,30925,12105], 11362)
# # replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [10695,58822,30925], 11362)
# # replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [10695,58822,12105], 11362)
# print(df_6_valid.value_counts('hosp_id_base'))
# print(df_12_valid.value_counts('hosp_id_base'))
# replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [12865,60529], 11029)

# # replace_values([df_6_valid,df_12_valid,df_6_regression,df_12_regression], [12865,60529,11868], 11029)

In [52]:
#添加结局糖化和BMI
df_v_copy=df_v.copy()
df_v_copy['visit_at'] = pd.to_datetime(df_v_copy['visit_at']).dt.date
df_v_copy['user_id'] = df_v_copy['user_id'].astype(float)
# df_v_copy.loc[:, 'visit_at'] = pd.to_datetime(df_v_copy['visit_at'], errors='coerce')
df_v_copy.rename(columns={'visit_at': 'target_Hb_date'}, inplace=True)
# print(df_v_copy[df_v_copy['user_id']==1264655.0])
df_v_copy.head()

,user_id,visit_level,target_Hb_date,hosp_id,age,diabetic_duration,sbp,dbp,bmi_v1,ALT,LDL_c,Cr,FPG,CP0,HbA1c
0,20071.0,1,2022-10-25,10695,67,290.0,132.0,72.0,22.1,55.3,2.50,68.9,6.71,2.072,8.1
1,854897.0,24,2022-01-12,10695,55,161.0,NaN,NaN,NaN,18.3,3.07,52.8,8.43,NaN,6.9
2,854897.0,26,2022-09-20,10695,56,169.0,147.0,91.0,22.5,15.0,1.78,52.0,9.60,4.220,8.0
7,1133103.0,32,2024-05-13,11362,44,89.0,124.0,78.0,24.9,NaN,NaN,NaN,8.80,1.933,11.6
8,1181708.0,17,2021-01-22,11362,35,46.0,150.0,93.0,25.6,27.0,2.37,66.0,7.30,NaN,6.3


In [53]:
#从df2_first_statistics中补充'sex_base', 'eGFR_EPI_base'
def merge_columns(df1, df2, keys=['user_id', 'type_no_class', 'first_Hb_date'], columns_to_add=['sex_base', 'eGFR_EPI_base']):
    # 使用merge函数根据A, B, C列匹配，将df2中的M, N列补充到df1中
    df1=df1.copy()
    df2=df2.copy()
    # 确保匹配列的数据类型一致
    for key in keys:
        df1[key] = df1[key].astype(str).str.strip()
        df2[key] = df2[key].astype(str).str.strip()

# 去除 df2 中在 keys 列组合上的重复值，保留第一个出现的
    df2_unique = df2.drop_duplicates(subset=keys)

    # 使用merge函数根据A, B, C列匹配，将df2中的M, N列补充到df1中
    df1_updated = pd.merge(df1, df2_unique[keys + columns_to_add], on=keys, how='left')


    return df1_updated

In [59]:
#临时，输出数据
df_6_valid1 = merge_columns(df_6_valid, df_v_copy, keys=['user_id', 'target_Hb_date'], columns_to_add=['bmi_v1', 'HbA1c'])
df_6_valid1['diabetic_duration_base'] = df_6_valid1['diabetic_duration_base'] / 12
df_6_valid1.rename(columns={'bmi_v1': 'bmi_end','HbA1c': 'HbA1c_end'}, inplace=True)
print(df_6_valid1.shape)
print(df_6_valid1[ 'bmi_end'].isnull().sum())
df_6_valid1.to_csv(os.path.join(datapath, 'df_6_valid_all_information.csv'), index=False)
df_6_valid1.head()

df_12_valid1 = merge_columns(df_12_valid, df_v_copy, keys=['user_id', 'target_Hb_date'], columns_to_add=['bmi_v1', 'HbA1c'])
df_12_valid1['diabetic_duration_base'] = df_12_valid1['diabetic_duration_base'] / 12
df_12_valid1.rename(columns={'bmi_v1': 'bmi_end','HbA1c': 'HbA1c_end'}, inplace=True)
print(df_12_valid1.shape)
print(df_12_valid1[ 'bmi_end'].isnull().sum())
df_12_valid1.to_csv(os.path.join(datapath, 'df_12_valid_all_information.csv'), index=False)
df_12_valid1.head()

# df_6_regression1 = merge_columns(df_6_regression, df_v_copy, keys=['user_id', 'target_Hb_date'], columns_to_add=['bmi_v1', 'HbA1c'])
# df_6_regression1['diabetic_duration_base'] = df_6_regression1['diabetic_duration_base'] / 12
# df_6_regression1.rename(columns={'bmi_v1': 'bmi_end','HbA1c': 'HbA1c_end'}, inplace=True)
# print(df_6_regression1.shape)
# print(df_6_regression1[ 'bmi_end'].isnull().sum())
# df_6_regression1.head()

# df_12_regression1 = merge_columns(df_12_regression, df_v_copy, keys=['user_id', 'target_Hb_date'], columns_to_add=['bmi_v1', 'HbA1c'])
# df_12_regression1['diabetic_duration_base'] = df_12_regression1['diabetic_duration_base'] / 12
# df_12_regression1.rename(columns={'bmi_v1': 'bmi_end','HbA1c': 'HbA1c_end'}, inplace=True)
# print(df_12_regression1.shape)
# print(df_12_regression1[ 'bmi_end'].isnull().sum())
# df_12_regression1.head()



(1185, 26)
284
(912, 26)
242


,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,cumi_flag,...,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag,bmi_end,HbA1c_end
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2019-06-20,2022-12-05,11362.0,331,1,...,24.7,10.0,2.79,31.9,10.82,0.868,9.4,0,24.8,8.9
1,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2021-04-06,2021-04-06,11362.0,277,1,...,21.1,10.1,2.01,116.8,8.06,3.541,7.7,0,21.0,7.3
2,1243003.0,2.0,2020-07-09,2023-09-01,2020-05-14,2021-04-28,2023-04-17,11362.0,293,0,...,23.6,16.2,2.92,84.9,8.63,1.408,7.0,1,23.1,6.7
3,1264655.0,1.0,2017-06-15,2020-03-25,2017-06-15,2018-06-04,2020-06-03,11362.0,354,0,...,33.4,10.9,2.11,86.1,4.82,2.213,7.1,0,29.0,8.6
4,1264976.0,2.0,2020-09-04,2021-06-23,2020-09-04,2021-04-28,2021-04-28,11362.0,236,1,...,29.3,55.7,2.56,45.9,12.71,2.610,7.7,0,26.7,8.1


In [4]:

# 使用方法
df2_first_statistics=pd.read_csv(os.path.join(datapath, 'df2_first_statistics.csv'))
df_valid1 = merge_columns(df_valid, df2_first_statistics)
df_valid1['diabetic_duration_base'] = df_valid1['diabetic_duration_base'] / 12
print(df_valid1.shape)
print('user_id:',df_valid1['user_id'].nunique())
df_valid1.head()

df_6_valid1 = merge_columns(df_6_valid, df2_first_statistics)
df_6_valid1['diabetic_duration_base'] = df_6_valid1['diabetic_duration_base'] / 12
print(df_6_valid1.shape)
print('user_id:',df_6_valid1['user_id'].nunique())
df_6_valid1.head()

df_12_valid1 = merge_columns(df_12_valid, df2_first_statistics)
df_12_valid1['diabetic_duration_base'] = df_12_valid1['diabetic_duration_base'] / 12
print(df_12_valid1.shape)
print('user_id:',df_12_valid1['user_id'].nunique())
df_12_valid1.head()

(3299, 26)
user_id: 2207
(1802, 26)
user_id: 1643
(1497, 26)
user_id: 1261


,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,cumi_flag,...,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag,sex_base,eGFR_EPI_base
0,1211166.0,2.0,2018-07-24,2022-12-05,2018-05-18,2019-06-20,2022-12-05,11362,331,1,...,24.7,10.0,2.79,31.9,10.82,0.868,9.4,0,2.0,132.90
1,1240517.0,2.0,2020-07-03,2021-01-09,2020-04-16,2021-04-06,2021-04-06,11362,277,1,...,21.1,10.1,2.01,116.8,8.06,3.541,7.7,0,2.0,41.60
2,1243003.0,2.0,2020-07-09,2023-09-01,2020-05-14,2021-04-28,2023-04-17,11362,293,0,...,23.6,16.2,2.92,84.9,8.63,1.408,7.0,1,1.0,86.14
3,1264655.0,1.0,2017-06-15,2020-03-25,2017-06-15,2018-06-04,2020-06-03,11362,354,0,...,33.4,10.9,2.11,86.1,4.82,2.213,7.1,0,2.0,61.86
4,1264976.0,2.0,2020-09-04,2021-06-23,2020-09-04,2021-04-28,2021-04-28,11362,236,1,...,29.3,55.7,2.56,45.9,12.71,2.610,7.7,0,2.0,115.45


In [5]:
import scipy.stats as stats
def summarize_and_compare(dataframe, type_col='type_no_class'):
    dataframe=dataframe.copy()
    dataframe.loc[:, 'type_no_class'] = pd.to_numeric(dataframe['type_no_class'], errors='coerce')
    # 分组
    group1 = dataframe[dataframe[type_col] == 1]
    group2 = dataframe[dataframe[type_col] == 2]

    # 初始化结果存储
    results = {'Feature': [], 'Overall_Mean': [], 'Overall_Std': [],
               'Mean_Group1': [],  'Std_Group1': [],
               'Mean_Group2': [],'Std_Group2': [],
              'P_Value': [],'NonMissing_Group1': [], 'NonMissing_Group2': []}

    # 遍历除type_col以外的所有列
    for column in dataframe.columns:
        if column != type_col:
            mean1 = round(group1[column].mean(), 2)
            mean2 = round(group2[column].mean(), 2)
            std1 = round(group1[column].std(), 1)
            std2 = round(group2[column].std(), 1)

            # 计算两组之间的p值
            t_stat, p_val = stats.ttest_ind(group1[column], group2[column], nan_policy='omit')
            p_val = round(p_val, 3)
            # print(f'{column}: Group1 Mean: {mean1}, Group2 Mean: {mean2}, P-Value: {p_val}')
            # 计算非缺失值个数和比例
            non_missing_count1 = group1[column].notna().sum()
            non_missing_count2 = group2[column].notna().sum()
            total_count1 = len(group1)
            total_count2 = len(group2)
            proportion1 = round((non_missing_count1 / total_count1) * 100, 1)
            proportion2 = round((non_missing_count2 / total_count2) * 100, 1)
            # print(f'{column}: count1: {non_missing_count1}, count2: {non_missing_count2}, proportion1: {proportion1}%, proportion2: {proportion2}%')
            non_missing_info1 = f"{non_missing_count1}({proportion1}%)"
            non_missing_info2 = f"{non_missing_count2}({proportion2}%)"

            # 计算整体均值和标准差（不分组）
            overall_mean = round(dataframe[column].mean(), 2)
            overall_std = round(dataframe[column].std(), 1)

            # 存储结果
            results['Feature'].append(column)
            results['Mean_Group1'].append(mean1)
            results['Mean_Group2'].append(mean2)
            results['Std_Group1'].append(std1)
            results['Std_Group2'].append(std2)
            results['P_Value'].append(p_val)
            results['NonMissing_Group1'].append(non_missing_info1)
            results['NonMissing_Group2'].append(non_missing_info2)
            results['Overall_Mean'].append(overall_mean)
            results['Overall_Std'].append(overall_std)

        # 转换为DataFrame返回
    result_df = pd.DataFrame(results)
    #换算为百分比
  # 将前四行的均值列转换为百分比，并加上百分号
    for i in range(min(4, len(result_df))):
        result_df.at[i, 'Mean_Group1'] = f"{int(result_df.at[i, 'Mean_Group1'] * 100)}%"
        result_df.at[i, 'Mean_Group2'] = f"{int(result_df.at[i, 'Mean_Group2'] * 100)}%"
        result_df.at[i, 'Overall_Mean'] = f"{int(result_df.at[i, 'Overall_Mean'] * 100)}%"
    
    # 将倒数第三行的均值列先减去1后转换为百分比，并加上百分号
    if len(result_df) >= 3:
        index = len(result_df) - 3
        result_df.at[index, 'Mean_Group1'] = f"{int((result_df.at[index, 'Mean_Group1'] - 1) * 100)}%"
        result_df.at[index, 'Mean_Group2'] = f"{int((result_df.at[index, 'Mean_Group2'] - 1) * 100)}%"
        result_df.at[index, 'Overall_Mean'] = f"{int((result_df.at[index, 'Overall_Mean'] - 1) * 100)}%"

    return result_df

In [6]:
#增加对四种药之一的统计
df_valid2 = df_valid1.drop(['T0','T1','first_Hb_date','target_Hb_date','last_Hb_date','hosp_id_base','drug_valid_time','normal_Hb_flag'], axis=1)
def add_column_E(df1):
    # 检查 A, B, C, D 四列的值是否全为0
    df1['Oral_hypoglycemic'] = df1[['cumi_flag', 'jia_flag', 'tang_flag', 'TZD_flag']].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
    return df1
df_valid2=add_column_E(df_valid2)
df_valid2.head()

df_6_valid2 = df_6_valid1.drop(['T0','T1','first_Hb_date','target_Hb_date','last_Hb_date','hosp_id_base','drug_valid_time','normal_Hb_flag'], axis=1)
def add_column_E(df1):
    # 检查 A, B, C, D 四列的值是否全为0
    df1['Oral_hypoglycemic'] = df1[['cumi_flag', 'jia_flag', 'tang_flag', 'TZD_flag']].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
    return df1
df_6_valid2=add_column_E(df_6_valid2)
df_6_valid2.head()

df_12_valid2 = df_12_valid1.drop(['T0','T1','first_Hb_date','target_Hb_date','last_Hb_date','hosp_id_base','drug_valid_time','normal_Hb_flag'], axis=1)
def add_column_E(df1):
    # 检查 A, B, C, D 四列的值是否全为0
    df1['Oral_hypoglycemic'] = df1[['cumi_flag', 'jia_flag', 'tang_flag', 'TZD_flag']].apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
    return df1
df_12_valid2=add_column_E(df_12_valid2)
df_12_valid2.head()

,user_id,type_no_class,cumi_flag,jia_flag,tang_flag,TZD_flag,age_base,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,sex_base,eGFR_EPI_base,Oral_hypoglycemic
0,1211166.0,2.0,1,1,1,0,42.0,5.000000,59.0,24.7,10.0,2.79,31.9,10.82,0.868,9.4,2.0,132.90,1
1,1240517.0,2.0,1,1,0,0,67.0,19.250000,64.0,21.1,10.1,2.01,116.8,8.06,3.541,7.7,2.0,41.60,1
2,1243003.0,2.0,0,1,0,0,59.0,10.666667,80.0,23.6,16.2,2.92,84.9,8.63,1.408,7.0,1.0,86.14,1
3,1264655.0,1.0,0,1,1,0,63.0,8.083333,86.0,33.4,10.9,2.11,86.1,4.82,2.213,7.1,2.0,61.86,1
4,1264976.0,2.0,1,1,1,0,45.0,3.250000,91.0,29.3,55.7,2.56,45.9,12.71,2.610,7.7,2.0,115.45,1


In [7]:
#仅统计患者人数，不统计患者条数
df_valid2.drop_duplicates(subset=['user_id'],keep='first',inplace=True)
df_valid3 = df_valid2.drop(['user_id'], axis=1)

df_6_valid2.drop_duplicates(subset=['user_id'],keep='first',inplace=True)
df_6_valid3 = df_6_valid2.drop(['user_id'], axis=1)

df_12_valid2.drop_duplicates(subset=['user_id'],keep='first',inplace=True)
df_12_valid3 = df_12_valid2.drop(['user_id'], axis=1)

In [8]:
#1代表GLP，2代表SGLT
print(df_valid3['type_no_class'].value_counts())
print(df_6_valid3['type_no_class'].value_counts())
# df_valid3.to_csv(os.path.join(datapath, 'table1_data.csv'))
# result_valid=summarize_and_compare(df_valid3)
# result_valid.to_csv(os.path.join(datapath, 'result_valid.csv'))
df_6_valid3.to_csv(os.path.join(datapath, 'table1_6_data.csv'))
result_6_valid=summarize_and_compare(df_6_valid3)
result_6_valid.to_csv(os.path.join(datapath, 'result_6_valid.csv'))
result_6_valid

type_no_class
2.0    1570
1.0     637
Name: count, dtype: int64
type_no_class
2.0    1161
1.0     482
Name: count, dtype: int64


,Feature,Overall_Mean,Overall_Std,Mean_Group1,Std_Group1,Mean_Group2,Std_Group2,P_Value,NonMissing_Group1,NonMissing_Group2
0,cumi_flag,21%,0.4,20%,0.4,22%,0.4,0.284,482(100.0%),1161(100.0%)
1,jia_flag,80%,0.4,83%,0.4,78%,0.4,0.044,482(100.0%),1161(100.0%)
2,tang_flag,40%,0.5,42%,0.5,40%,0.5,0.408,482(100.0%),1161(100.0%)
3,TZD_flag,5%,0.2,7%,0.3,5%,0.2,0.049,482(100.0%),1161(100.0%)
4,age_base,52.57,12.9,50.12,13.3,53.59,12.6,0.000,482(100.0%),1161(100.0%)
5,diabetic_duration_base,6.98,7.7,7.79,8.3,6.64,7.4,0.006,482(100.0%),1161(100.0%)
6,dbp_base,79.76,11.2,80.11,11.6,79.61,11.1,0.415,482(100.0%),1161(100.0%)
7,bmi_v1_base,26.86,4.1,28.45,4.1,26.2,3.9,0.000,482(100.0%),1161(100.0%)
8,ALT_base,33.4,24.5,37.72,27.5,31.61,23.0,0.000,482(100.0%),1161(100.0%)
9,LDL_c_base,3.03,1.0,2.97,1.0,3.05,1.0,0.120,482(100.0%),1161(100.0%)


In [9]:
print(df_12_valid3['type_no_class'].value_counts())
# df_valid3.to_csv(os.path.join(datapath, 'table1_data.csv'))
# result_valid=summarize_and_compare(df_valid3)
# result_valid.to_csv(os.path.join(datapath, 'result_valid.csv'))
df_12_valid3.to_csv(os.path.join(datapath, 'table1_12_data.csv'))
result_12_valid=summarize_and_compare(df_12_valid3)
result_12_valid.to_csv(os.path.join(datapath, 'result_12_valid.csv'))
result_12_valid

type_no_class
2.0    921
1.0    340
Name: count, dtype: int64


,Feature,Overall_Mean,Overall_Std,Mean_Group1,Std_Group1,Mean_Group2,Std_Group2,P_Value,NonMissing_Group1,NonMissing_Group2
0,cumi_flag,23%,0.4,19%,0.4,25%,0.4,0.022,340(100.0%),921(100.0%)
1,jia_flag,81%,0.4,86%,0.3,79%,0.4,0.007,340(100.0%),921(100.0%)
2,tang_flag,43%,0.5,41%,0.5,44%,0.5,0.297,340(100.0%),921(100.0%)
3,TZD_flag,5%,0.2,5%,0.2,5%,0.2,0.609,340(100.0%),921(100.0%)
4,age_base,52.27,12.6,49.44,12.9,53.32,12.3,0.000,340(100.0%),921(100.0%)
5,diabetic_duration_base,7.42,7.6,8.08,8.3,7.17,7.3,0.062,340(100.0%),921(100.0%)
6,dbp_base,80.07,11.1,80.89,10.6,79.76,11.3,0.107,340(100.0%),921(100.0%)
7,bmi_v1_base,26.88,4.0,28.88,4.0,26.14,3.8,0.000,340(100.0%),921(100.0%)
8,ALT_base,31.91,23.5,37.24,26.7,29.94,21.9,0.000,340(100.0%),921(100.0%)
9,LDL_c_base,2.96,1.1,2.91,1.0,2.97,1.1,0.333,340(100.0%),921(100.0%)


In [17]:
import pickle

def compute_and_save_folds(data, num_folds, save_path):
    # 计算每个hosp_id的数据量
    hosp_id_counts = data['hosp_id_base'].value_counts().sort_values(ascending=False)
    
    # 初始化分折存储结构
    folds = {i: [] for i in range(num_folds)}
    fold_sizes = [0] * num_folds
    
    # 动态分配hosp_id到folds
    for hosp_id, count in hosp_id_counts.items():
        min_fold_idx = fold_sizes.index(min(fold_sizes))
        folds[min_fold_idx].append(hosp_id)
        fold_sizes[min_fold_idx] += count
    fold_data_sizes = []
    total_data_count = data.shape[0]
    for i in range(num_folds):
        fold_hosp_ids = folds[i]
        fold_data = data[data['hosp_id_base'].isin(fold_hosp_ids)]
        fold_data_sizes.append(fold_data.shape[0])
    print(f"Fold data sizes: {fold_data_sizes}", f"Total data count: {total_data_count}")
    # 保存分配结果
    with open(save_path, 'wb') as f:
        pickle.dump(folds, f)
    
    return folds

In [18]:
def save_fold(df_valid,name,k=10):
    df_valid1=df_valid[df_valid['type_no_class'] ==1]
    df_valid2=df_valid[df_valid['type_no_class'] ==2]
    folds_valid=compute_and_save_folds(df_valid, k, os.path.join(datapath, 'ten_folds/all_')+str(name)+'.pkl')
    folds_valid1=compute_and_save_folds(df_valid1, k, os.path.join(datapath, 'ten_folds/GLP_')+str(name)+'.pkl')
    folds_valid2=compute_and_save_folds(df_valid2, k, os.path.join(datapath, 'ten_folds/SGLT_')+str(name)+'.pkl')
    print()

In [19]:
# df_3_valid=pd.read_csv(os.path.join(datapath, 'df_3_valid.csv'))
df_6_valid=pd.read_csv(os.path.join(datapath, 'df_6_valid.csv'))
df_12_valid=pd.read_csv(os.path.join(datapath, 'df_12_valid.csv'))
# df_3_regression=pd.read_csv(os.path.join(datapath, 'df_3_regression.csv'))
df_6_regression=pd.read_csv(os.path.join(datapath, 'df_6_regression.csv'))
df_12_regression=pd.read_csv(os.path.join(datapath, 'df_12_regression.csv'))
# save_fold(df_3_valid,'3_valid')
save_fold(df_6_valid,'6_valid')
save_fold(df_12_valid,'12_valid')
# save_fold(df_3_regression,'3_regression')
save_fold(df_6_regression,'6_regression')
save_fold(df_12_regression,'12_regression')

Fold data sizes: [1764, 1764, 1764, 1764, 1764, 1764, 1764, 1763, 1763, 1763] Total data count: 17637
Fold data sizes: [411, 411, 411, 411, 411, 411, 411, 411, 411, 411] Total data count: 4110
Fold data sizes: [1353, 1353, 1353, 1353, 1353, 1353, 1353, 1352, 1352, 1352] Total data count: 13527

Fold data sizes: [1731, 1731, 1731, 1731, 1731, 1731, 1731, 1731, 1731, 1731] Total data count: 17310
Fold data sizes: [372, 372, 372, 372, 372, 372, 372, 372, 372, 372] Total data count: 3720
Fold data sizes: [1359, 1359, 1359, 1359, 1359, 1359, 1359, 1359, 1359, 1359] Total data count: 13590

Fold data sizes: [1780, 1780, 1780, 1780, 1780, 1780, 1779, 1779, 1779, 1779] Total data count: 17796
Fold data sizes: [414, 414, 414, 414, 414, 414, 414, 414, 414, 414] Total data count: 4140
Fold data sizes: [1366, 1366, 1366, 1366, 1366, 1366, 1365, 1365, 1365, 1365] Total data count: 13656

Fold data sizes: [1731, 1731, 1731, 1731, 1731, 1731, 1731, 1730, 1730, 1730] Total data count: 17307
Fold data 

In [20]:
df_6_valid.head()

,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_valid_time,cumi_flag,...,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,normal_Hb_flag
0,10238.0,1.0,2020-01-10,2020-05-17,2019-10-24,2020-07-24,2020-07-24,20006.0,196,0,...,177.0,67.794512,29.540316,35.0,1.40,98.0,12.38,1.970000,7.1,0
1,902628.0,2.0,2018-12-24,2023-09-01,2018-12-17,2019-03-26,2022-08-23,20006.0,92,1,...,71.0,100.000000,27.600000,23.0,3.58,67.0,15.25,2.080000,9.8,1
2,902628.0,2.0,2018-12-24,2023-09-01,2018-12-17,2019-06-25,2022-08-23,20006.0,183,1,...,71.0,100.000000,27.600000,23.0,3.58,67.0,15.25,2.080000,9.8,1
3,985157.0,1.0,2019-05-11,2019-08-13,2019-05-07,2019-08-13,2019-08-13,20006.0,94,0,...,120.0,67.000000,34.300000,21.0,3.55,62.0,6.72,1.318246,8.0,0
4,985248.0,1.0,2016-10-31,2017-02-17,2016-10-31,2017-02-17,2017-02-17,20006.0,109,1,...,81.0,75.000000,30.300000,64.0,3.25,60.0,10.62,-0.132678,8.0,1


In [21]:
df_6_regression.head()

,user_id,type_no_class,T0,T1,first_Hb_date,target_Hb_date,last_Hb_date,hosp_id_base,drug_regression_time,cumi_flag,...,diabetic_duration_base,dbp_base,bmi_v1_base,ALT_base,LDL_c_base,Cr_base,FPG_base,CP0_base,HbA1c_base,HbA1c_xx_month
0,10238.0,1.0,2020-01-10,2020-05-17,2019-10-24,2020-07-24,2020-07-24,20006.0,196,0,...,177.0,82.830421,32.15269,35.0,1.40,98.0,12.38,1.970000,7.1,7.8
1,902628.0,2.0,2018-12-24,2023-09-01,2018-12-17,2019-03-26,2022-08-23,20006.0,92,1,...,71.0,100.000000,27.60000,23.0,3.58,67.0,15.25,2.080000,9.8,6.8
2,902628.0,2.0,2018-12-24,2023-09-01,2018-12-17,2019-06-25,2022-08-23,20006.0,183,1,...,71.0,100.000000,27.60000,23.0,3.58,67.0,15.25,2.080000,9.8,8.2
3,985157.0,1.0,2019-05-11,2019-08-13,2019-05-07,2019-08-13,2019-08-13,20006.0,94,0,...,120.0,67.000000,34.30000,21.0,3.55,62.0,6.72,5.221367,8.0,7.7
4,985248.0,1.0,2016-10-31,2017-02-17,2016-10-31,2017-02-17,2017-02-17,20006.0,109,1,...,81.0,75.000000,30.30000,64.0,3.25,60.0,10.62,2.230904,8.0,6.2
